In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from matplotlib.ticker import PercentFormatter
from scipy import stats as st
from scipy.stats import norm, beta
import matplotlib.ticker as ticker
import re
import math
#from statsmodels.stats.outliers_influence import variance_inflation_factor 
from statsmodels.tools.tools import add_constant
from statsmodels.regression.linear_model import OLS
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
####CHECK STORED DF#####
# Deserialization
with open("dfXY.pickle", "rb") as infile:
    dfXYPickle = pickle.load(infile)
#print("Reconstructed object", dfXYPickle)

In [ ]:
dfXY = dfXYPickle.copy()

In [ ]:
print(dfXY.shape)
dfXY.head()

In [ ]:
#dfXY.dtypes

In [ ]:
dfXYold=dfXY.copy()

In [ ]:
#Remove steelgradeid
dfSteelGradeID=dfXY.SteelGradeID.copy()
dfXY.drop("SteelGradeID",inplace=True,axis=1)

In [ ]:
#Drop RWP sensors: not in use
dfRWP=dfXY[['AP1_FurnaceRWPEntryTemp','AP1_FurnaceRWPExitTemp']].copy()
dfXY.drop(['AP1_FurnaceRWPEntryTemp','AP1_FurnaceRWPExitTemp'],axis=1,inplace=True)

In [ ]:
#Check for nan
dfXY.isna().sum().sum()

In [ ]:
#Numeric vars
numerical_var_series=dfXY[dfXY.columns[dfXY.dtypes == np.float64]].columns#.to_list()#[0:5]
print(type(numerical_var_series))
print(len(numerical_var_series))
numerical_var_series

In [ ]:
#Create the different groups of columns to plot
proceso_vars_series=dfXY[numerical_var_series[(~numerical_var_series.str.contains("AP1_Furnace",regex=False)) & (~numerical_var_series.str.contains("pyro2",regex=False))]].columns#.head(1)
zoneTemp_vars_series=dfXY[numerical_var_series[numerical_var_series.str.contains(".*Zone.*Temp.*",regex=True)]].columns#.head(1)

zoneAirGas_vars_series=dfXY[numerical_var_series[numerical_var_series.str.contains(".*Zone.*Air.*|.*Zone.*Gas.*",regex=True)]].columns#.head(1)

recuperator_vars_series=dfXY[numerical_var_series[numerical_var_series.str.contains("AP1_Furnace.*Dilution|Recuperator|RWP|Combustion.*",regex=True)]].columns#.head(1)

pyro_vars_series=dfXY[numerical_var_series[numerical_var_series.str.contains(".*pyro.*|.*Pyro.*",regex=True)]].columns#.head(1)

rest_vars_series=dfXY[[var for var in numerical_var_series.to_list() if var not in proceso_vars_series.to_list() and var not in zoneTemp_vars_series.to_list() and var not in zoneAirGas_vars_series.to_list() and var not in recuperator_vars_series.to_list() and var not in pyro_vars_series.to_list()]].columns#.head(1)

In [ ]:
#Zone1 air SP are scaled with outdated scale, rescale:outdated scale: 47565 SCFH->updated scale: 59655 SCFH
dfXY.loc[:,zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains(".*Zone1.*AirFlowSet[Pp]oint.*",regex=True)]]=dfXY.loc[:,zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains(".*Zone1.*AirFlowSet[Pp]oint.*",regex=True)]]*47565/59655

In [ ]:
#Zone2 air SP are scaled with outdated scale, rescale:outdated scale: 47565 SCFH->updated scale: 59655 SCFH
dfXY.loc[:,zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains(".*Zone2.*AirFlowSet[Pp]oint.*",regex=True)]]=dfXY.loc[:,zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains(".*Zone2.*AirFlowSet[Pp]oint.*",regex=True)]]*47565/59655

dfXY=dfXYold.copy()

## Remove outliers

#Make Ratio outliers nan to not loose data
dfXY.loc[:,dfXY.columns[dfXY.columns.str.contains("Ratio",regex=False)]] = dfXY.loc[:,dfXY.columns[dfXY.columns.str.contains("Ratio",regex=False)]].map(lambda x: x if ((x>=0) & (x<=4)) else np.nan,na_action='ignore')
#Check Ratios below 4
dfXY[dfXY.columns[dfXY.columns.str.contains("Ratio",regex=False)]].gt(4).sum(axis=0)
#Check now nans where ratios were above 4
dfXY[dfXY.columns[dfXY.columns.str.contains("Ratio",regex=False)]].isna().sum(axis=0)

In [ ]:
dfXY_description=dfXY.describe()
dfXY_description

In [ ]:
#Cuts to consider outliers
cut_neg = dfXY_description.iloc[1] - 3*dfXY_description.iloc[2] #mean-3*std
cut_pos = dfXY_description.iloc[1] + 3*dfXY_description.iloc[2] #mean+3*std
cut_neg.name = 'cut-'
cut_pos.name = 'cut+'
cut_neg = pd.DataFrame(cut_neg).T
cut_pos = pd.DataFrame(cut_pos).T
tbl_desc = pd.concat([dfXY_description,cut_neg, cut_pos])
tbl_desc

In [ ]:
#Tune cut
for c in range(tbl_desc.shape[1]):
    if tbl_desc.loc['cut-',tbl_desc.columns[c]]<0:
        tbl_desc.loc['cut-',tbl_desc.columns[c]]=0
    if tbl_desc.columns[c]=='AP1_FurnaceWidth':
        tbl_desc.loc['cut-',tbl_desc.columns[c]]=0 #some widthIDs are smaller than the actual cut-
    elif tbl_desc.columns[c]=='NetWeight':
        tbl_desc.loc['cut-',tbl_desc.columns[c]]=5000 #min around 20k, if cut for defects would be 10k. To be conservative 5k
    elif tbl_desc.columns[c] in tbl_desc.columns[tbl_desc.columns.str.contains('Ratio',regex=False)]:
        tbl_desc.loc['cut-',tbl_desc.columns[c]]=0
        tbl_desc.loc['cut+',tbl_desc.columns[c]]=4
    elif tbl_desc.columns[c] in tbl_desc.columns[tbl_desc.columns.str.contains("P1_FurnaceZone.*[C].*[V].*",regex=True)]:
        tbl_desc.loc['cut-',tbl_desc.columns[c]]=0
        tbl_desc.loc['cut+',tbl_desc.columns[c]]=100 #CV vars are in %
    elif  tbl_desc.columns[c] in zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains("Zone0",regex=False) & ~zoneAirGas_vars_series.str.contains("Ratio",regex=False)]:
        tbl_desc.loc['cut-',tbl_desc.columns[c]]=0
        tbl_desc.loc['cut+',tbl_desc.columns[c]]=100 #Zone0 airgas vars are in %
    elif tbl_desc.columns[c] in zoneTemp_vars_series[~zoneTemp_vars_series.str.contains("CV",regex=False)]:
        if tbl_desc.loc['cut-',tbl_desc.columns[c]]<=500:
            tbl_desc.loc['cut-',tbl_desc.columns[c]]=500 #Zone 0 seems to have many values near 0 that are outside the bell
    elif tbl_desc.columns[c] in pyro_vars_series[pyro_vars_series.str.contains("Temp",regex=False)]:
        tbl_desc.loc['cut-',tbl_desc.columns[c]]=1400 #pyro sensors seem to have many values at 1200 that are outside the bimodal bell, p() bc it is the lower limit of the sensor
tbl_desc

In [ ]:
#Remove outliers
for var in tbl_desc.columns:
    dfXY.loc[:,var]=dfXY.loc[:,var].map(lambda x: x if ((x>=tbl_desc.loc["cut-",var]) & (x<=tbl_desc.loc["cut+",var])) else np.nan,na_action='ignore')
dfXY.isna().sum(axis=0)

In [ ]:
#Remove outliers when LineSpeed=0
for t in dfXY.index:
    if (dfXY.loc[t,"AP1_FurnaceLineSpeed"]==0) | (np.isnan(dfXY.loc[t,"AP1_FurnaceLineSpeed"])):
        dfXY.loc[t,dfXY.columns[~dfXY.columns.str.contains("AP1_FurnaceLineSpeed")]] = np.nan
dfXY.isna().sum(axis=0)

## Create sort of SteelFamID: SteelFamCluster

In [ ]:
modelGMM = GaussianMixture(n_components=2).fit(dfXY.pyro2.dropna().to_numpy().reshape(-1,1))
clustersGMM = modelGMM.predict(dfXY.pyro2.dropna().to_numpy().reshape(-1,1))
print(np.unique(clustersGMM))
c=0
fig,ax = plt.subplots(1,1,figsize=(10,4),facecolor='white')
fig.suptitle("Clustering of pyro2goals with GMM",fontsize=20)
#Compute df
clusteringGMM_Long = pd.DataFrame(dfXY.pyro2,columns=['pyro2'])#.reset_index(drop=True)
for t in dfXY.index:
    if np.isnan(clusteringGMM_Long.loc[t,"pyro2"]):
        clusteringGMM_Long.loc[t,"SteelFamCluster"] = np.nan
    else:
        clusteringGMM_Long.loc[t,"SteelFamCluster"] = clustersGMM[c]
        c+=1
clusteringGMM=pd.merge(right=clusteringGMM_Long.drop_duplicates().reset_index(drop=True),left=pd.DataFrame(dfXY.pyro2.value_counts()).reset_index(), how='inner',on='pyro2').sort_values(by=['pyro2'])
  
#Plor barplot
sns.barplot(data=clusteringGMM,x='pyro2',y='count',hue='SteelFamCluster',palette="Set2",width=.2,ax=ax)#,ax=ax[1]
#Add labels
barRectangles = ax.patches
labels = clusteringGMM['count'].values
label_i=0
#Cannot loop through zip(barRectangles,labels) bc barRect is larger as there are rect in the bins with 0 counts
for rect in barRectangles:#zip()stop when the shortest list ends, iterate through rect bars
    height = rect.get_height()
    if height != 0:
        #print(rect.get_x(),height+6,label)
        ax.text(x=rect.get_x()+rect.get_width()/2,y=height+6,s=str(int(height)),ha="center",va="bottom",color="black",fontsize=10)#s=str(labels[label_i])
        label_i+=1

#Make room for all labels in plot
ax.set_ylim(top=max(labels)+1500)
#Other way, but not centered horizontally as u dont have the width of the bars
#for index, value in dfXY.pyro2.value_counts().sort_index().items():
#    axes[r,c].text(x=index, y=value, s=value, color='yellow', ha='center')

fig.tight_layout(pad=3)
clusteringGMM

In [ ]:
#Assign meaningfull labels to clusters
a=pd.DataFrame(modelGMM.means_,columns=["center"]).assign(label = lambda x: x.iloc[:,0].map(lambda y: "HighGoal" if y==x.max()[0] else "LowGoal")).reset_index(names="cluster")
a

In [ ]:
#Check mapping works before applying to df
# Defining all the conditions for labelling
def condition(y):
    if y==a.loc[a.label=="HighGoal","cluster"].values[0]:
        return "HighGoal"
    elif np.isnan(y):
        return np.nan
    elif y==a.loc[a.label=="LowGoal","cluster"].values[0]:
        return "LowGoal"
    else:
        pass
clusteringGMM_Long.assign(labelsGMM = lambda x: x.SteelFamCluster.map(condition).astype("category")).drop_duplicates()
#pd.DataFrame(clustersGMM,columns=['clustersGMM'],index=dfXY.index,dtype=np.int64).map(lambda y: "HighGoal" if y==a.loc[a.label=="HighGoal","cluster"][0] else "LowGoal").astype("category").copy()

In [ ]:
#Print pyro2 clusters with new label
fig,ax = plt.subplots(1,1,figsize=(10,4))
fig.suptitle("Clustering of pyro2goals with GMM",fontsize=20)
#Compute df
clusteringGMM=clusteringGMM.assign(labelsGMM = lambda x: x.SteelFamCluster.map(condition).astype("category"))
#Plor barplot
sns.barplot(data=clusteringGMM,x='pyro2',y='count',hue='labelsGMM',palette="Set2",width=.2,ax=ax)#,ax=ax[1]
#Add labels
barRectangles = ax.patches
labels = clusteringGMM['count'].values
label_i=0
#Cannot loop through zip(barRectangles,labels) bc barRect is larger as there are rect in the bins with 0 counts
for rect in barRectangles:#zip()stop when the shortest list ends, iterate through rect bars
    height = rect.get_height()
    if height != 0:
        #print(rect.get_x(),height+6,label)
        ax.text(x=rect.get_x()+rect.get_width()/2,y=height+6,s=str(int(height)),ha="center",va="bottom",color="black",fontsize=10)#s=str(labels[label_i])
        label_i+=1

#Make room for all labels in plot
ax.set_ylim(top=max(labels)+1500)
fig.tight_layout(pad=3)

In [ ]:
#Add clusters to dfXY
#if dfXY.SteelFamCluster.shape: dfXY.drop("SteelFamCluster",axis=1,inplace=True)
dfXY.loc[:,'SteelFamCluster']=clusteringGMM_Long.SteelFamCluster.map(condition).astype("category")
dfXY=dfXY.copy()
dfXY.SteelFamCluster.dtypes

In [ ]:
dfXY.head()

## Create sort of WidthFamilyID: WidthFamCluster

In [ ]:
WidthFamCluster = GaussianMixture(n_components=2,random_state=1).fit(dfXY.CurrentWidth.dropna().to_numpy().reshape(-1,1)).predict(dfXY.CurrentWidth.dropna().to_numpy().reshape(-1,1))
print(np.unique(WidthFamCluster))
c=0
fig,ax = plt.subplots(1,1,figsize=(10,5),facecolor='white')
fig.suptitle("Clustering of WidthFamilyID with GMM",fontsize=20)
#Compute df
clusteringGMM = pd.DataFrame(dfXY.CurrentWidth,columns=['CurrentWidth'])#.reset_index(drop=True)
for t in dfXY.index:
 #clusteringGMM=pd.concat([pd.DataFrame(dfXY.CurrentWidth.dropna(),columns=['CurrentWidth']).reset_index(drop=True),pd.DataFrame(WidthFamCluster,columns=['WidthFamCluster'])],axis=1)
    if np.isnan(clusteringGMM.loc[t,"CurrentWidth"]):
        clusteringGMM.loc[t,"WidthFamCluster"] = np.nan
    else:
        clusteringGMM.loc[t,"WidthFamCluster"] = WidthFamCluster[c]
        c+=1
    #Plor histogram
sns.histplot(data=clusteringGMM,x='CurrentWidth',hue='WidthFamCluster',palette="Set2",ax=ax,kde=True)#,ax=ax[1]

fig.tight_layout(pad=3)
print(dfXY.shape,clusteringGMM.shape)
print(clusteringGMM.WidthFamCluster.unique())
clusteringGMM

In [ ]:
#Add clusters to dfXY
dfXY.loc[:,'WidthFamCluster']=clusteringGMM[["WidthFamCluster"]].astype("category")
dfXY=dfXY.copy()
dfXY.WidthFamCluster.dtypes

In [ ]:
dfXY.head(1)

## Create rampFdbk, soakFdbk, CSdeltaTmin features

In [ ]:
#CSdeltaT:Cross Section Time in Fce
#iterate timestamps of dfXY, not dfXY_coils
v0=0
time = dfXY.index[0]
dfXY['CSdeltaTmin'] = np.nan
frow0=0
for i,t in enumerate(dfXY.index): #compute CS t of coil in Fce
    if (i==0) & (dfXY.AP1_FurnaceLineSpeed.loc[t] == 0): #First row:
        frow0=1
        
    if (dfXY.AP1_FurnaceLineSpeed.loc[t] == 0) & (frow0==1):
        dfXY.loc[t,'CSdeltaTmin']=np.nan #dont know how much time will remain v=0
    elif (dfXY.AP1_FurnaceLineSpeed.loc[t] != 0) & (frow0==1):
        frow0=0
        
    if (dfXY.AP1_FurnaceLineSpeed.loc[t] == 0) & (v0==0) & (frow0!=1): #CS t inf
        v0=1
        time = t #update timeindex when v=0
    elif (v0==1) & (dfXY.AP1_FurnaceLineSpeed.loc[t] != 0):
       dfXY.loc[time:dfXY.index[i-1],'CSdeltaTmin'] = (dfXY.index[i]-time).total_seconds()/60 #if (dfXY.index[i-1]-time).total_seconds()/60 when v=0 only 1row, CSdeltaTmin=0.0 
       dfXY.loc[t,'CSdeltaTmin']=160/dfXY.loc[t,'AP1_FurnaceLineSpeed']
       v0=0
    elif (v0==0) & (dfXY.AP1_FurnaceLineSpeed.loc[t] != 0): 
        dfXY.loc[t,'CSdeltaTmin']=160/dfXY.loc[t,'AP1_FurnaceLineSpeed'] 
        
    if i == dfXY.shape[0]-1: #Last row
        if (v0==1) & (dfXY.AP1_FurnaceLineSpeed.loc[t] == 0):
            dfXY.loc[t,'CSdeltaTmin']=np.nan #dont know how much time will remain v=0 
#dfXY.CSdeltaTmin
dfXY=dfXY.copy()
dfXY.head(1)

In [ ]:
dfXY.CSdeltaTmin.describe()

In [ ]:
#Check nan values in CSdeltaTmin
print("Nan in CSdeltaTmin:", dfXY.CSdeltaTmin.isna().sum())
#Check inf values in CSdeltaTmin
print("inf in CSdeltaTmin:",dfXY.CSdeltaTmin[dfXY.CSdeltaTmin.isin([np.inf,-np.inf])].count())
#Check Linespeed 0: inf CSdeltaTmin are bc LineSpeed=0
print("Rows linespeed=0:", dfXY[dfXY.AP1_FurnaceLineSpeed==0].AP1_FurnaceLineSpeed.count())
#Check coils when linespeed 0
print("unique coils when linespeed 0:",dfXY_coils.loc[dfXY[dfXY.AP1_FurnaceLineSpeed==0].index].unique())
#Change inf by nan
# dfXY.CSdeltaTmin=dfXY.CSdeltaTmin.map(lambda x: np.nan if np.isinf(x) else x,na_action='ignore')
# dfXY.CSdeltaTmin[dfXY.CSdeltaTmin.isin([np.inf,-np.inf])].count()
#Check statistical measure values in CSdeltaTmin
dfXY[dfXY.AP1_FurnaceLineSpeed==0][['AP1_FurnaceLineSpeed','CSdeltaTmin']]

In [ ]:
#Coil Time in Fce: len dfXY_coils!=dfXY bc of grades mismatches
#iterate timestamps of dfXY, not dfXY_coils
coil = dfXY_coils.loc[dfXY.index[0]]
time = dfXY.index[0]
dfCoilInFce = pd.DataFrame(dfXY_coils.loc[dfXY.index].copy())
dfCoilInFce['TFce'] = 0 
dfCoilInFce.TFce = pd.to_timedelta(dfCoilInFce.TFce)#dataframe with coil number and t of coil in fce
for i,t in enumerate(dfXY.index): #compute t of coil in fce
    if dfCoilInFce.AP1_FurnaceCoilID.loc[t] != coil: #new coil entered furnace
        dfCoilInFce.loc[time:dfCoilInFce.index[i-1],'TFce'] = dfCoilInFce.index[i-1]-time
        coil = dfCoilInFce.AP1_FurnaceCoilID.loc[t] #update coil in furnace
        time = t #update timeindex when actual coil enters furnace
    if i == dfCoilInFce.shape[0]-1: #Last row
        dfCoilInFce.loc[time:dfCoilInFce.index[i],'TFce'] = dfCoilInFce.index[i]-time

dfCoilInFce.drop_duplicates()           

coil = dfXY_coils.unique()[6]#6,4,5,7,8,9,10,11,13,14,15,...
dfcoil=pd.DataFrame(dfXY_coils)[pd.DataFrame(dfXY_coils).AP1_FurnaceCoilID==coil]
dfcoil
#dfXY.loc[dfcoil.index[0]:dfcoil.index[len(dfcoil.index)-1],[dfXY.columns[dfXY.columns.str.contains('.*Zone.*Temp.*Setpoint',regex=True)].values.tolist()]] 
TempFdbk=pd.melt(dfXY.loc[dfcoil.index[0]:dfcoil.index[len(dfcoil.index)-1],dfXY.columns.str.contains('.*Zone.TopTemp|AP1_FurnaceZone0MasterTempFdbk',regex=True)].reset_index(),id_vars=['ts'])
TempSP=pd.melt(dfXY.loc[dfcoil.index[0]:dfcoil.index[len(dfcoil.index)-1],dfXY.columns.str.contains('.*Zone.*Temp.*Setpoint',regex=True)].reset_index(),id_vars=['ts'])
#Plot
fig, ax = plt.subplots(2,1,figsize=(20,15),facecolor='white')
fig.suptitle("Zone Temp",fontsize=20)
#SP
ax[0].set_title("SP for {}".format(coil),fontsize=10)
sns.lineplot(data=TempSP,x='variable',y='value',estimator=np.median,ax=ax[0],lw=1,errorbar='sd')#,hue='variable',estimator=np.median
sns.scatterplot(data=TempSP,x='variable',y='value',ax=ax[0],markers={'s':20,'marker':'o'},hue='ts')#,hue='variable'
#Fbk
ax[1].set_title("Fdbk for {}".format(coil),fontsize=10)
sns.lineplot(data=TempFdbk,x='variable',y='value',ax=ax[1],estimator=np.median,lw=1,errorbar='sd')#,hue='variable'
sns.scatterplot(data=TempFdbk,x='variable',y='value',ax=ax[1],markers={'s':20,'marker':'o'},hue='ts')#,hue='variable'
#dfXY.loc[dfcoil.index[0]:dfcoil.index[len(dfcoil.index)-1],dfXY.columns.str.contains('.*Zone.TopTemp|AP1_FurnaceZone0MasterTempFdbk',regex=True)]
#dfXY.loc[dfcoil.index[0]:dfcoil.index[len(dfcoil.index)-1],dfXY.columns.str.contains('.*Zone.*Temp.*Setpoint',regex=True)]

In [ ]:
#Iterate all coils and count weird/davay distributions (include TurndownOn var to see if its related):len dfXY_coils!=dfXY bc of grades mismatches
#iterate timestamps of dfXY, not dfXY_coils
coil = dfXY_coils.loc[dfXY.index[0]]
time = dfXY.index[0]
tot=1
weirdList = []#(initime,endtime,coil,TurndownOn.bool)
davayList = []#(initime,endtime,coil,TurndownOn.bool)
for i,t in enumerate(dfXY.index):
    if dfXY_coils.loc[t] != coil: #new coil entered furnace
        #Long-format DF SP & Fdbk
        TempFdbk=pd.melt(dfXY.loc[time:dfXY.index[i-1],dfXY.columns.str.contains('.*Zone.TopTemp|AP1_FurnaceZone0MasterTempFdbk',regex=True)].reset_index(),id_vars=['ts'])
        TempSP=pd.melt(dfXY.loc[time:dfXY.index[i-1],dfXY.columns.str.contains('.*Zone.*Temp.*Setpoint',regex=True)].reset_index(),id_vars=['ts'])
        
        if (TempFdbk.groupby(['variable']).mean('value').min()==TempFdbk.groupby(['variable']).mean('value').iloc[0]).value: #if shape not weird (Z0 is min)
            #Check if reason of weird shape is TurndownOn variable
            TurndownOn = dfXY.loc[time:dfXY.index[i-1],dfXY.columns.str.contains('.*Zone.*TurndownOn',regex=True)].astype(np.float64).sum(axis=0).sum()>0#.apply(pd.unique,axis=0)
            davayList.append((time,dfXY.index[i-1],coil,TurndownOn))
            
        else: 
            #Check if reason of weird shape is TurndownOn variable
            TurndownOn = dfXY.loc[time:dfXY.index[i-1],dfXY.columns.str.contains('.*Zone.*TurndownOn',regex=True)].astype(np.float64).sum(axis=0).sum()>0#.apply(pd.unique,axis=0)
            weirdList.append((time,dfXY.index[i-1],coil,TurndownOn))
        
        tot+=1
        coil = dfXY_coils.loc[t] #update coil in furnace
        time = t #update timeindex when actual coil enters furnace
        
    if i == dfXY.shape[0]-1: #Last row
        #Long-format DF SP & Fdbk
        TempFdbk=pd.melt(dfXY.loc[time:dfXY.index[i],dfXY.columns.str.contains('.*Zone.TopTemp|AP1_FurnaceZone0MasterTempFdbk',regex=True)].reset_index(),id_vars=['ts'])
        TempSP=pd.melt(dfXY.loc[time:dfXY.index[i],dfXY.columns.str.contains('.*Zone.*Temp.*Setpoint',regex=True)].reset_index(),id_vars=['ts'])

        if (TempFdbk.groupby(['variable']).mean('value').min()==TempFdbk.groupby(['variable']).mean('value').iloc[0]).value: #if shape not weird (Z0 is min)
            #Check if reason of weird shape is TurndownOn variable
            TurndownOn = dfXY.loc[time:dfXY.index[i],dfXY.columns.str.contains('.*Zone.*TurndownOn',regex=True)].astype(np.float64).sum(axis=0).sum()>0#.apply(pd.unique,axis=0)
            davayList.append((time,dfXY.index[i],coil,TurndownOn))
        else: 
            #Check if reason of weird shape is TurndownOn variable
            TurndownOn = dfXY.loc[time:dfXY.index[i],dfXY.columns.str.contains('.*Zone.*TurndownOn',regex=True)].astype(np.float64).sum(axis=0).sum()>0#.apply(pd.unique,axis=0)
            weirdList.append((time,dfXY.index[i],coil,TurndownOn))
        
if tot == len(weirdList)+len(davayList):
    print("There are {} weird temp distributions and {} davay distributions, which make a total of {}".format(len(weirdList),len(davayList),tot))
    print("There are {} unique coils".format(len(dfXY_coils.loc[dfXY.index].unique())))
else:
    print("WARNING: There are {} weird temp distributions and {} davay distributions, which make a total of {}".format(len(weirdList),len(davayList),len(weirdList)+len(davayList)))
    print("There are {} total temp distributions".format(tot))

#Print ex
fig,ax=plt.subplots(2,1,figsize=(20,10),facecolor='white')
fig.suptitle('Example of temp distributions',fontsize=20)
#Print an example of weird distribution
coil = weirdList[1]#first tuple (tini,tend,coil)
TempFdbkWeird=pd.melt(dfXY.loc[coil[0]:coil[1],dfXY.columns.str.contains('.*Zone.TopTemp|AP1_FurnaceZone0MasterTempFdbk',regex=True)].reset_index(),id_vars=['ts'])
ax[0].set_title("Fdbk of weird temp distribution for coil {}, with TurndownOn {}".format(coil[2],coil[3]),fontsize=10)
sns.lineplot(data=TempFdbkWeird,x='variable',y='value',ax=ax[0],estimator=np.mean,lw=1,errorbar='sd')#,hue='variable'
sns.scatterplot(data=TempFdbkWeird,x='variable',y='value',ax=ax[0],markers={'s':20,'marker':'o'},hue='ts')#,hue='variable'
ax[0].set_ylabel("Temp",fontsize=10,labelpad=15)
ax[0].set_xlabel("Variable",fontsize=10,labelpad=15)
ax[0].tick_params('x', labelrotation=45)
#Print an example of davay distribution
coil = davayList[1]#first tuple (tini,tend,coil)
TempFdbkDavay=pd.melt(dfXY.loc[coil[0]:coil[1],dfXY.columns.str.contains('.*Zone.TopTemp|AP1_FurnaceZone0MasterTempFdbk',regex=True)].reset_index(),id_vars=['ts'])
ax[1].set_title("Fdbk of davay temp distribution for coil {}, with TurndownOn {}".format(coil[2],coil[3]),fontsize=10)
sns.scatterplot(data=TempFdbkDavay,x='variable',y='value',ax=ax[1],markers={'s':20,'marker':'o'},hue='ts')#,hue='variable'
#sns.scatterplot(data=TempFdbkDavay.groupby("variable").mean().reset_index(),x='variable',y='value',ax=ax[1],marker='x',size=300,color='fuchsia')#,hue='variable','c':'black',markers={'s':80,'marker':'x'}
sns.lineplot(data=TempFdbkDavay,x='variable',y='value',ax=ax[1],estimator=np.mean,lw=1,errorbar='sd',markers=True,marker='x')#,hue='variable',color='red'
ax[1].set_ylabel("Temp",fontsize=10,labelpad=15)#,markers={'s':80,'marker':'x','color':'red'}
ax[1].set_xlabel("Variable",fontsize=10,labelpad=15)
#plt.xticks(rotation=45)
ax[1].tick_params('x', labelrotation=45)
fig.tight_layout(pad=3)
plt.savefig(r"C:\Users\js5296\Desktop\NAS\Projects\AP1_Furnace_Control\EDAs\new\MV\newTempRampSoak.png", bbox_inches="tight")

In [ ]:
#SHOW TEM DISTRIBUTION FOR SP
#sns.scatterplot(data=TempFdbkDavay.groupby("variable").mean().reset_index(),x='variable',y='value',marker='x',size=200,color='fuchsia',markers={'s':200,'marker':'x'})#,hue='variable','c':'black'
coil = davayList[2]
TempSPDavay=pd.melt(dfXY.loc[coil[0]:coil[1],dfXY.columns.str.contains('.*Zone.*Temp.*Setpoint',regex=True)].reset_index(),id_vars=['ts'])
#TempFdbkDavay=pd.melt(dfXY.loc[coil[0]:coil[1],dfXY.columns.str.contains('.*Zone.TopTemp|AP1_FurnaceZone0MasterTempFdbk',regex=True)].reset_index(),id_vars=['ts'])
fig,ax=plt.subplots(1,1,figsize=(20,10),facecolor='white')
fig.suptitle('Example of tempSP distributions',fontsize=20)
ax.set_title("SP of temp distribution for coil {}, with TurndownOn {}".format(coil[2],coil[3]),fontsize=10)
ax.set_ylabel("Temp",fontsize=10,labelpad=15)
ax.set_xlabel("Variable",fontsize=10,labelpad=15)
sns.lineplot(data=TempSPDavay,x='variable',y='value',ax=ax,estimator=np.mean,lw=1,errorbar='sd')#,hue='variable'
sns.scatterplot(data=TempSPDavay,x='variable',y='value',ax=ax,markers={'s':20,'marker':'o'},hue='ts')#,hue='variable'
plt.xticks(rotation=45)
fig.tight_layout(pad=3)

In [ ]:
#SHOW TEM DISTRIBUTION AVERAGED BY STEELGRADEFAM (2 DISTRIBUTIONS): USE IT TO CREATE rampFdbk/soakFdbk VARIABLES
#TempFdbk=pd.melt(dfXY.loc[time:dfXY.index[i-1],dfXY.columns.str.contains('.*Zone.TopTemp|AP1_FurnaceZone0MasterTempFdbk',regex=True)].reset_index(),id_vars=['ts'])
#dfXY[dfXY.columns[dfXY.columns.str.contains('.*Zone.TopTemp|AP1_FurnaceZone0MasterTempFdbk|SteelFamCluster',regex=True)]].groupby("SteelFamCluster").std().reset_index()
fig,axes=plt.subplots(figsize=(20,7))
#sns.lineplot(data=pd.melt(dfXY[dfXY.columns[dfXY.columns.str.contains('.*Zone.TopTemp|AP1_FurnaceZone0MasterTempFdbk|SteelFamCluster',regex=True)]].groupby("SteelFamCluster").mean().reset_index(),id_vars=['SteelFamCluster']),x='variable',y='value',hue='SteelFamCluster',lw=1,marker='o',errorbar='sd',markers=True,palette="dark:#5A9_r",ax=axes)#markers={'s':20,'marker':'o'}
dfRampSoak = dfXY[dfXY.columns[dfXY.columns.str.contains('.*Zone.TopTemp|AP1_FurnaceZone0MasterTempFdbk|SteelFamCluster',regex=True)]]#.groupby("SteelFamCluster").mean().reset_index()
sns.lineplot(data=pd.melt(dfRampSoak,id_vars=['SteelFamCluster']),x='variable',y='value',hue='SteelFamCluster',estimator=np.mean,err_style="band",errorbar='sd',markers=True,lw=1.5,marker='o',palette="dark:#5A9_r",ax=axes)#markers={'s':20,'marker':'o'}
fig.suptitle('Temp distributions by SteelFamCluster',fontsize=30)
axes.tick_params(axis='x', labelsize=10)
axes.tick_params(axis='y', labelsize=10)
axes.set_xlabel("VARIABLE",fontsize=10,labelpad=10)
axes.set_ylabel("VALUE",fontsize=10,labelpad=15)
plt.xticks(rotation=45)
fig.tight_layout(pad=3)
plt.savefig(r"C:\Users\js5296\Desktop\NAS\Projects\AP1_Furnace_Control\EDAs\new\MV\newAvgTempRampSoak.png", bbox_inches="tight")

#Add variables to dfXY
dfRampSoak = dfRampSoak.groupby("SteelFamCluster").mean().reset_index()
dfRampSoak = dfXY[dfXY.columns[dfXY.columns.str.contains('.*Zone.TopTemp|AP1_FurnaceZone0MasterTempFdbk|SteelFamCluster',regex=True)]].groupby("SteelFamCluster").mean()#.reset_index()
dfRampSoak["rampFdbk"] = dfRampSoak.apply(lambda x: x["AP1_FurnaceZone1TopTemp"]-x["AP1_FurnaceZone0MasterTempFdbk"],axis=1)
dfRampSoak["soakFdbk"] = dfRampSoak.iloc[:,1:-2].mean(axis=1)

dfXY.loc[:,"rampFdbk"] = dfXY.loc[:,"SteelFamCluster"].cat.rename_categories({"HighGoal":1,"LowGoal":0}).astype(float)
dfXY.loc[:,'rampFdbk']=dfXY[["rampFdbk"]].map(lambda y: dfRampSoak.loc["HighGoal","rampFdbk"] if y==1 else dfRampSoak.loc["LowGoal","rampFdbk"]) 

dfXY.loc[:,"soakFdbk"] = dfXY.loc[:,"SteelFamCluster"].cat.rename_categories({"HighGoal":1,"LowGoal":0}).astype(float)
dfXY.loc[:,'soakFdbk']=dfXY[["soakFdbk"]].map(lambda y: dfRampSoak.loc["HighGoal","soakFdbk"] if y==1 else dfRampSoak.loc["LowGoal","soakFdbk"])
dfRampSoak

## Create TurndownOn compressed column

In [ ]:
TurnDownOn = dfXY[dfXY.columns[dfXY.columns.str.contains('.*TurndownOn',regex=True)]].copy()
TurnDownOnCompressed=(dfXY[dfXY.columns[dfXY.columns.str.contains('.*TurndownOn',regex=True)]]==1).sum(axis=1).astype(np.int64).map(lambda x: int(1) if (x>0) else int(x),na_action='ignore').astype("category")
dfXY.drop(dfXY.columns[dfXY.columns.str.contains('.*TurndownOn',regex=True)],axis=1,inplace=True)
dfXY = pd.merge(left=dfXY,right=pd.DataFrame(TurnDownOnCompressed,columns=['TurnDownOnCompressed'],dtype="category"),how='inner',right_index=True,left_index=True)
dfXY.head(1)

In [ ]:
#Check no duplicated values introduced
dfXY.isna().sum(axis=0)
dfXY.index.duplicated().sum()

## Final outlier detection

In [ ]:
#Remove outliers when LineSpeed=0
for t in dfXY.index:
    if (dfXY.loc[t,"AP1_FurnaceLineSpeed"]==0) | (np.isnan(dfXY.loc[t,"AP1_FurnaceLineSpeed"])):
        dfXY.loc[t,dfXY.columns[~dfXY.columns.str.contains("AP1_FurnaceLineSpeed")]] = np.nan
dfXY.isna().sum(axis=0)

In [ ]:
CSdeltaTmin_describe=pd.DataFrame(dfXY.CSdeltaTmin.describe())
CSdeltaTmin_describe.loc["cut-","CSdeltaTmin"]=CSdeltaTmin_describe.loc["mean","CSdeltaTmin"]-3*CSdeltaTmin_describe.loc["std","CSdeltaTmin"]
CSdeltaTmin_describe.loc["cut+","CSdeltaTmin"]=CSdeltaTmin_describe.loc["mean","CSdeltaTmin"]+3*CSdeltaTmin_describe.loc["std","CSdeltaTmin"]
CSdeltaTmin_describe
tbl_desc.loc[:,"CSdeltaTmin"]=CSdeltaTmin_describe
tbl_desc

In [ ]:
#Tune cut
for c in range(tbl_desc.shape[1]):
    if tbl_desc.loc['cut-',tbl_desc.columns[c]]<0:
        tbl_desc.loc['cut-',tbl_desc.columns[c]]=0
    if tbl_desc.columns[c]=='AP1_FurnaceWidth':
        tbl_desc.loc['cut-',tbl_desc.columns[c]]=0 #some widthIDs are smaller than the actual cut-
    elif tbl_desc.columns[c]=='NetWeight':
        tbl_desc.loc['cut-',tbl_desc.columns[c]]=5000 #min around 20k, if cut for defects would be 10k. To be conservative 5k
    elif tbl_desc.columns[c] in tbl_desc.columns[tbl_desc.columns.str.contains('Ratio',regex=False)]:
        tbl_desc.loc['cut-',tbl_desc.columns[c]]=0
        tbl_desc.loc['cut+',tbl_desc.columns[c]]=4
    elif tbl_desc.columns[c] in tbl_desc.columns[tbl_desc.columns.str.contains("P1_FurnaceZone.*[C].*[V].*",regex=True)]:
        tbl_desc.loc['cut-',tbl_desc.columns[c]]=0
        tbl_desc.loc['cut+',tbl_desc.columns[c]]=100 #CV vars are in %
    elif  tbl_desc.columns[c] in zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains("Zone0",regex=False) & ~zoneAirGas_vars_series.str.contains("Ratio",regex=False)]:
        tbl_desc.loc['cut-',tbl_desc.columns[c]]=0
        tbl_desc.loc['cut+',tbl_desc.columns[c]]=100 #Zone0 airgas vars are in %
    elif tbl_desc.columns[c] in zoneTemp_vars_series[~zoneTemp_vars_series.str.contains("CV",regex=False)]:
        if tbl_desc.loc['cut-',tbl_desc.columns[c]]<=500:
            tbl_desc.loc['cut-',tbl_desc.columns[c]]=500 #Zone 0 seems to have many values near 0 that are outside the bell
    elif tbl_desc.columns[c] in pyro_vars_series[pyro_vars_series.str.contains("Temp",regex=False)]:
        tbl_desc.loc['cut-',tbl_desc.columns[c]]=1400 #pyro sensors seem to have many values at 1200 that are outside the bimodal bell, p() bc it is the lower limit of the sensor
tbl_desc

In [ ]:
#Remove outliers from CSdeltaTmin
dfXY.loc[:,"CSdeltaTmin"]=dfXY.loc[:,"CSdeltaTmin"].map(lambda x: x if ((x>=tbl_desc.loc["cut-","CSdeltaTmin"]) & (x<=tbl_desc.loc["cut+","CSdeltaTmin"])) else np.nan,na_action='ignore')
dfXY.isna().sum(axis=0)

## Group analysis

In [ ]:
#Numeric vars
numerical_var_series=dfXY[dfXY.columns[dfXY.dtypes == np.float64]].columns#.to_list()#[0:5]
print(type(numerical_var_series))
print(len(numerical_var_series))
numerical_var_series

In [ ]:
#Categorical vars
categorical_var=dfXY[dfXY.columns[dfXY.dtypes == "category"]].columns
print(type(categorical_var))
print(len(categorical_var))
categorical_var

In [ ]:
proceso_vars_series

In [ ]:
# dfXY_mx=dfXY.corr()
# sns.heatmap(dfXY_mx, annot=True, cmap='Blues')

In [ ]:
dfXY=np.round(dfXY,2)
dfXY.head(1)

In [ ]:
#Define func avg zones of furnace
def func_get_dataReg(data:pd.DataFrame):# Expects [pyro2,pyro2Fdbk,zones], where data=df[pyro,pyroFdbk,zones].groupby(pyro2).mean().reset_index()
    dataF=data.iloc[:,[0,1,2]].rename(columns={data.columns[2]:'X'})
    for i in range(3,data.shape[1]):
        data1=data.iloc[:,[0,1,i]].rename(columns={data.columns[i]:'X'})
        dataF=pd.concat([dataF,data1],axis=0)
    return dataF.groupby("pyro2").mean().reset_index()#Returns the mean value of all vars (avg the furnace zones) for each pyro2goal [pyro2,pyro2Fdbk,X].

In [ ]:
#Define func avg Bottom\Top by Zone
def func_avg_zone(l:list):#Averages Top/Bottom values for each zone
    df=pd.DataFrame(np.zeros((dfXY.shape[0],len(l)//2)))
    df.iloc[:,:]=np.nan
    i=0
    j=0
    while i < len(l)//2:#j<9
        df.iloc[:,i]=dfXY.loc[:,[l[j],l[j+1]]].mean(axis=1)
        #print("{} vs {}".format(l[j],l[j+1]))
        j+=2
        i+=1
    df=pd.concat([df.reset_index(drop=True),pd.DataFrame(dfXY[["AP1_FurnacePyro2Temp","grade","TurnDownOnCompressed","WidthFamCluster","SteelFamCluster"]].reset_index(drop=True))],axis=1)
    df.columns=["Zone0","Zone1","Zone2","Zone3","Zone4","Zone5","Zone6","Zone7","Zone8","AP1_FurnacePyro2Temp","grade","TurnDownOnCompressed","WidthFamCluster","SteelFamCluster"]
    return df

if None:
    print("None")
    
if ["hdgsiu","skudgs"]:
    print("List")
a=pd.DataFrame({"A": [1, 2, 3], "B": [4, 5, 6]}).rename(columns={"A":"Peepe"})[["Peepe","B","B"]]
a.loc[:,~a.columns.duplicated()]
a.loc[:,["Peepe"]]#.columns.duplicated()

In [ ]:
#Define func plot regression
def func_plot_reg(nrow:int,ncol:int,ylist:list,x1list:list,x2list:list=None,ylabel:str=None,x1label:str=None,x2label:str=None,Zone:bool=False,Avg:bool=False,totlist:list=None,title:str="scatter",kde:bool=False):
    if totlist:
        tot=totlist
        T=True
    else: 
        T=False 
        tot=[0]#no tot plotted
    t=0
    col=ncol
    row=nrow
    #row=(len(zone)//col+1)+2#+2 for the 2 scatterplots of SP/Fdbk of all zones integrated
    hue = ["grade","TurnDownOnCompressed","WidthFamCluster","SteelFamCluster"]
    X1 = True
    
    zone=["Zone0","Zone1","Zone2","Zone3","Zone4","Zone5","Zone6","Zone7","Zone8"]
    z=0
    
    #Initialize values
    scatter=dfXY[ylist+x1list]
    reg=dfXY[ylist+x1list]
    scatter2=dfXY[ylist+x1list]
    reg2=dfXY[ylist+x1list]
    onevsone=False
    onevsn=False
    n1vsm_ZA=False

    #Check if its 1vs1, 1vsn noZone or 1vsn Zone
    if Zone & Avg:#nvsm,1vsn Zone & Avg
        scatter=pd.concat([pd.DataFrame(dfXY[ylist+["pyro2"]].reset_index(drop=True)),func_avg_zone(x1list).iloc[:,:]],axis=1)
        scatter=scatter.loc[:,~scatter.columns.duplicated()]
        reg=pd.concat([pd.DataFrame(dfXY[["pyro2"]+ylist].reset_index(drop=True)),func_avg_zone(x1list).iloc[:,[0,1,2,3,4,5,6,7,8,9]]],axis=1)
        reg=reg.loc[:,~reg.columns.duplicated()].groupby("pyro2").mean().reset_index()#.iloc[:,[0,19]+list(range(1,19))]
        n1vsm_ZA=True
        if len(ylist)==1:
            scatter.rename(columns={ylist[0]:"Y"},inplace=True)
            reg.rename(columns={ylist[0]:"Y"},inplace=True)
        if x2list:
            scatter2=pd.concat([pd.DataFrame(dfXY[ylist+["pyro2"]].reset_index(drop=True)),func_avg_zone(x2list).iloc[:,:]],axis=1)
            scatter2=scatter2.loc[:,~scatter2.columns.duplicated()]
            reg2=pd.concat([pd.DataFrame(dfXY[["pyro2"]+ylist].reset_index(drop=True)),func_avg_zone(x2list).iloc[:,[0,1,2,3,4,5,6,7,8,9]]],axis=1)
            reg2=reg2.loc[:,~reg2.columns.duplicated()].groupby("pyro2").mean().reset_index()#.iloc[:,[0,19]+list(range(1,19))]
            if len(ylist)==1:
                scatter2.rename(columns={ylist[0]:"Y"},inplace=True)
                reg2.rename(columns={ylist[0]:"Y"},inplace=True)
    elif (~Zone) & (x2list==None):#(1vs1, 1vsn) noZone
        if (len(ylist)==1) & (len(x1list)==1):#1vs1 noZone
            zone=hue.copy()
            onevsone=True
            reg=dfXY[["pyro2"]+ylist+x1list]
            reg=reg.loc[:,~reg.columns.duplicated()].groupby("pyro2").mean().reset_index().rename(columns={ylist[0]:"Y",x1list[0]:"X"})
        elif (len(ylist)>1) & (len(x1list)>1):#mean(m)vsn noZone (tempvsX)
            onevsn=True
            zone=x1list.copy()
            scatter=pd.concat([pd.DataFrame(dfXY[ylist].mean(axis=1),columns=["Y"]),dfXY[zone+hue]],axis=1)
            reg=pd.concat([pd.DataFrame(dfXY[ylist].mean(axis=1),columns=["Y"]),dfXY[zone+["pyro2"]]],axis=1).groupby("pyro2").mean().reset_index()#.iloc[:,[len(zone)-1+2]+list(range(len(zone)-1+2))].groupby(...
        elif (len(ylist)==1) & (len(x1list)>1):#1vsn (pyro2vsX)
            onevsn=True
            zone=x1list.copy()
            scatter=(dfXY[["pyro2"]+ylist+x1list+hue])
            scatter=scatter.loc[:,~scatter.columns.duplicated()].rename(columns={ylist[0]:"Y"})
            reg=(dfXY[["pyro2"]+ylist+x1list])
            reg=reg.loc[:,~reg.columns.duplicated()].groupby("pyro2").mean().reset_index().rename(columns={ylist[0]:"Y"})
            
    #Remove duplicated columns added twice
    scatter=scatter.loc[:,~scatter.columns.duplicated()]
    reg=reg.loc[:,~reg.columns.duplicated()]
    scatter2=scatter2.loc[:,~scatter2.columns.duplicated()]
    reg2=reg2.loc[:,~reg2.columns.duplicated()]
    
    fig,axes = plt.subplots(row,col,figsize=(20,(row*20/5)+2),facecolor='white')
    fig.suptitle("{} by {}".format(title,hue[3]),fontsize=30)
    for r in range(row):
        for c in range(col):
            if z<len(zone):
                if X1:
                    if n1vsm_ZA:#nvsm,1vsn Zone and Avg
                        if len(ylist)==1:
                            sns.scatterplot(data=scatter,x=zone[z],y="Y",ax=axes[r,c],hue=hue[3],palette="pastel")#,size=20
                            if kde & (scatter[zone[z]].std()!=0) & (scatter["Y"].std()!=0):
                                sns.kdeplot(data=scatter,x=zone[z],y="Y",levels=5,thresh=0.2,alpha=0.5,hue=hue[3],ax=axes[r,c],palette=["green","purple"])#,levels=5,thresh=0.2,palette="dark:#5A9_r
                            sns.regplot(data=reg,x=zone[z],y="Y",ax=axes[r,c],color='black',line_kws={'lw':2,'color':'fuchsia'},order=1,scatter=True,marker='x',scatter_kws={'s':40},x_ci=None,ci=None)#,marker='x',scatter_kws={'s':20}
                            if (len(reg.Y.unique())>1) & (len(reg[zone[z]].unique())>1):
                                nan = np.logical_or(reg.Y.isna().to_numpy(), reg[zone[z]].isna().to_numpy())#OR of na values in X and Y, as pearsonr doesnt like nans
                                axes[r,c].legend(['r\u00b2={:.2f}, p={:.2E}'.format(st.pearsonr(reg.Y[~nan],reg[zone[z]][~nan])[0]**2,st.pearsonr(reg.Y[~nan],reg[zone[z]][~nan])[1])],fontsize='large')
                        else:
                            sns.scatterplot(data=scatter,x=zone[z],y=ylist[z],ax=axes[r,c],hue=hue[3],palette="pastel")#,size=20
                            if kde & (scatter[zone[z]].std()!=0) & (scatter[ylist[z]].std()!=0):
                                sns.kdeplot(data=scatter,x=zone[z],y=ylist[z],levels=5,thresh=0.2,alpha=0.5,hue=hue[3],ax=axes[r,c],palette=["green","purple"])#,levels=5,thresh=0.2,palette="dark:#5A9_r
                            sns.regplot(data=reg,x=zone[z],y=ylist[z],ax=axes[r,c],color='black',line_kws={'lw':2,'color':'fuchsia'},order=1,scatter=True,marker='x',scatter_kws={'s':40},x_ci=None,ci=None)#,marker='x',scatter_kws={'s':20}
                            if (len(reg[ylist[z]].unique())>1) & (len(reg[zone[z]].unique())>1):
                                nan = np.logical_or(reg[ylist[z]].isna().to_numpy(), reg[zone[z]].isna().to_numpy())#OR of na values in X and Y, as pearsonr doesnt like nans
                                axes[r,c].legend(['r\u00b2={:.2f}, p={:.2E}'.format(st.pearsonr(reg[ylist[z]][~nan],reg[zone[z]][~nan])[0]**2,st.pearsonr(reg[ylist[z]][~nan],reg[zone[z]][~nan])[1])],fontsize='large')
                            
                    elif onevsn:#mean(Y)vsn or 1vsn
                        sns.scatterplot(scatter,x=zone[z],y="Y",ax=axes[r,c],hue=hue[3],palette="pastel")#,size=20
                        if kde & (scatter[zone[z]].std()!=0) & (scatter["Y"].std()!=0):
                            sns.kdeplot(data=scatter,x=zone[z],y="Y",levels=5,thresh=0.2,alpha=0.5,hue=hue[3],ax=axes[r,c],palette=["green","purple"])#,levels=5,thresh=0.2,palette="dark:#5A9_r
                        sns.regplot(data=reg,x=zone[z],y="Y",ax=axes[r,c],color='black',line_kws={'lw':2,'color':'fuchsia'},order=1,scatter=True,marker='x',scatter_kws={'s':40},x_ci=None,ci=None)#,marker='x',scatter_kws={'s':20}  
                        if (len(reg.Y.unique())>1) & (len(reg[zone[z]].unique())>1):
                            nan = np.logical_or(reg.Y.isna().to_numpy(), reg[zone[z]].isna().to_numpy())#OR of na values in X and Y, as pearsonr doesnt like nans
                            axes[r,c].legend(['r\u00b2={:.2f}, p={:.2E}'.format(st.pearsonr(reg.Y[~nan],reg[zone[z]][~nan])[0]**2,st.pearsonr(reg.Y[~nan],reg[zone[z]][~nan])[1])],fontsize='large')
                    elif onevsone:
                        sns.scatterplot(dfXY,x=x1list[0],y=ylist[0],ax=axes[r,c],hue=hue[z])#,size=20
                        #sns.kdeplot(data=scatter,x=x1list[0],y=ylist[0],levels=5,thresh=0.2,alpha=0.5,hue=hue[3],ax=axes[r,c],palette=["green","purple"])#,levels=5,thresh=0.2,palette="dark:#5A9_r
                        sns.regplot(data=reg,x="X",y="Y",ax=axes[r,c],color='black',line_kws={'lw':2,'color':'fuchsia'},order=1,scatter=True,marker='x',scatter_kws={'s':40},x_ci=None,ci=None)#,marker='x',scatter_kws={'s':20}  
                        if (len(reg.Y.unique())>1) & (len(reg.X.unique())>1):
                            nan = np.logical_or(reg.Y.isna().to_numpy(), reg.X.isna().to_numpy())#OR of na values in X and Y, as pearsonr doesnt like nans
                            axes[r,c].legend(['r\u00b2={:.2f}, p={:.2E}'.format(st.pearsonr(reg.Y[~nan],reg.X[~nan])[0]**2,st.pearsonr(reg.Y[~nan],reg.X[~nan])[1])],fontsize='large')
                    #axes[r,c].set_ylim(1000,2400)
                    #axes[r,c].set_title(label='Pyro2 vs {} by {}'.format(var[v],hue[3]),loc='center',pad=10,fontsize=20)
                    axes[r,c].set_title(label='{}'.format(zone[z]),loc='center',pad=10,fontsize=17)
                    axes[r,c].set_xlabel(xlabel=x1label,labelpad=10,fontsize='x-large')
                    axes[r,c].set_ylabel(ylabel=ylabel,labelpad=10,fontsize='x-large')

                    if x2list:
                        X1=False
                    else: z+=1
                else:#X2
                    if len(ylist)==1:
                        sns.scatterplot(data=scatter2,x=zone[z],y="Y",ax=axes[r,c],hue=hue[3],palette="pastel")#,size=20
                        if kde & (scatter[zone[z]].std()!=0) & (scatter["Y"].std()!=0):
                            sns.kdeplot(data=scatter,x=zone[z],y="Y",levels=5,thresh=0.2,alpha=0.5,hue=hue[3],ax=axes[r,c],palette=["green","purple"])#,levels=5,thresh=0.2,palette="dark:#5A9_r
                        sns.regplot(data=reg2,x=zone[z],y="Y",ax=axes[r,c],color='black',line_kws={'lw':2,'color':'fuchsia'},order=1,scatter=True,marker='x',scatter_kws={'s':40},x_ci=None,ci=None)#,marker='x',scatter_kws={'s':20}
                        if (len(reg2.Y.unique())>1) & (len(reg2[zone[z]].unique())>1):
                            nan = np.logical_or(reg2.Y.isna().to_numpy(), reg2[zone[z]].isna().to_numpy())#OR of na values in X and Y, as pearsonr doesnt like nans
                            axes[r,c].legend(['r\u00b2={:.2f}, p={:.2E}'.format(st.pearsonr(reg2.Y[~nan],reg2[zone[z]][~nan])[0]**2,st.pearsonr(reg2.Y[~nan],reg2[zone[z]][~nan])[1])],fontsize='large')
                    else:
                        sns.scatterplot(data=scatter2,x=zone[z],y=ylist[z],ax=axes[r,c],hue=hue[3],palette="pastel")#,size=20
                        if kde & (scatter[zone[z]].std()!=0) & (scatter[ylist[z]].std()!=0):
                            sns.kdeplot(data=scatter,x=zone[z],y=ylist[z],levels=5,thresh=0.2,alpha=0.5,hue=hue[3],ax=axes[r,c],palette=["green","purple"])#,levels=5,thresh=0.2,palette="dark:#5A9_r
                        sns.regplot(data=reg2,x=zone[z],y=ylist[z],ax=axes[r,c],color='black',line_kws={'lw':2,'color':'fuchsia'},order=1,scatter=True,marker='x',scatter_kws={'s':40},x_ci=None,ci=None)#,marker='x',scatter_kws={'s':20}
                        if (len(reg2[ylist[z]].unique())>1) & (len(reg2[zone[z]].unique())>1):
                            nan = np.logical_or(reg2[ylist[z]].isna().to_numpy(), reg2[zone[z]].isna().to_numpy())#OR of na values in X and Y, as pearsonr doesnt like nans
                            axes[r,c].legend(['r\u00b2={:.2f}, p={:.2E}'.format(st.pearsonr(reg2[ylist[z]][~nan],reg2[zone[z]][~nan])[0]**2,st.pearsonr(reg2[ylist[z]][~nan],reg2[zone[z]][~nan])[1])],fontsize='large')
                    #axes[r,c].set_title(label='Pyro2 vs {} by {}'.format(var[v],hue[3]),loc='center',pad=10,fontsize=20)
                    axes[r,c].set_title(label='{}'.format(zone[z]),loc='center',pad=10,fontsize=17)
                    axes[r,c].set_xlabel(xlabel=x2label,labelpad=10,fontsize='x-large')
                    axes[r,c].set_ylabel(ylabel=ylabel,labelpad=10,fontsize='x-large')
                    
                    X1=True
                    z+=1
                    
                axes[r,c].tick_params(axis='x', labelsize=15)
                axes[r,c].tick_params(axis='y', labelsize=15)
                #axes[r,c].set_ylim(1000,2400)
                #print(z>=len(zone),t<len(tot),T)
            elif ((z>=len(zone)) & (t<len(tot)) & T): #Print SP/Fdbk zones alltoguether

                #print("In Tot")
                if t==0:
                    scatter=pd.concat([pd.DataFrame(dfXY[ylist+["pyro2"]].reset_index(drop=True)),func_avg_zone(x1list).iloc[:,:]],axis=1)
                    scatter=scatter.loc[:,~scatter.columns.duplicated()]
                    for v,variable in enumerate(zone):
                        if len(ylist)>1:
                            sns.scatterplot(data=scatter,x=variable,y=ylist[v],ax=axes[r,c],hue=hue[3])#,size=2
                        else:
                            sns.scatterplot(data=scatter,x=variable,y=ylist[0],ax=axes[r,c],hue=hue[3])#,size=2
                    if len(ylist)>1:#Temp    
                        Y=pd.concat([pd.DataFrame(dfXY[["pyro2"]+ylist].reset_index(drop=True)),func_avg_zone(x1list).iloc[:,[0,1,2,3,4,5,6,7,8,9]]],axis=1)
                        Y=func_get_dataReg(Y.loc[:,~Y.columns.duplicated()].groupby("pyro2").mean().reset_index().iloc[:,[0,19]+list(range(1,19))].iloc[:,:11]).X
                        X=pd.concat([pd.DataFrame(dfXY[["pyro2"]+ylist].reset_index(drop=True)),func_avg_zone(x1list).iloc[:,[0,1,2,3,4,5,6,7,8,9]]],axis=1)
                        X=func_get_dataReg(X.loc[:,~X.columns.duplicated()].groupby("pyro2").mean().reset_index().iloc[:,[0,19]+list(range(1,19))].iloc[:,[0,1]+list(range(11,20))]).X
                    else:#pyro2
                        Y=pd.concat([pd.DataFrame(dfXY[["pyro2"]+ylist].reset_index(drop=True)),func_avg_zone(x1list).iloc[:,[0,1,2,3,4,5,6,7,8,9]]],axis=1)
                        Y=Y.loc[:,~Y.columns.duplicated()].groupby("pyro2").mean().reset_index().loc[:,ylist[0]]
                        X=pd.concat([pd.DataFrame(dfXY[["pyro2"]+ylist].reset_index(drop=True)),func_avg_zone(x1list).iloc[:,[0,1,2,3,4,5,6,7,8,9]]],axis=1)
                        X=func_get_dataReg(X.loc[:,~X.columns.duplicated()].groupby("pyro2").mean().reset_index().iloc[:,:]).X
                    sns.regplot(x=X,y=Y,ax=axes[r,c],color='black',line_kws={'lw':2,'color':'red'},order=1,scatter=True,marker='x',scatter_kws={'s':40},x_ci=None,ci=None)#,marker='x',scatter_kws={'s':20} 
                    if (len(X.unique())>1) & (len(Y.unique())>1):
                        nan = np.logical_or(Y.isna().to_numpy(), X.isna().to_numpy())#OR of na values in X and Y, as pearsonr doesnt like nans
                        axes[r,c].legend(['r\u00b2={:.2f}, p={:.2E}'.format(st.pearsonr(Y[~nan],X[~nan])[0]**2,st.pearsonr(Y[~nan],X[~nan])[1])],fontsize='medium')
                    else: axes[r,c].legend([hue[3]],loc='best')
                elif t==1:
                    scatter=pd.concat([pd.DataFrame(dfXY[ylist+["pyro2"]].reset_index(drop=True)),func_avg_zone(x2list).iloc[:,:]],axis=1)
                    scatter=scatter.loc[:,~scatter.columns.duplicated()]
                    for v,variable in enumerate(zone):
                        if len(ylist)>1:
                            sns.scatterplot(data=scatter,x=variable,y=ylist[v],ax=axes[r,c],hue=hue[3])#,size=2
                        else:
                            sns.scatterplot(data=scatter,x=variable,y=ylist[0],ax=axes[r,c],hue=hue[3])#,size=2
                    
                    if len(ylist)>1:#Temp    
                        Y=pd.concat([pd.DataFrame(dfXY[["pyro2"]+ylist].reset_index(drop=True)),func_avg_zone(x2list).iloc[:,[0,1,2,3,4,5,6,7,8,9]]],axis=1)
                        Y=func_get_dataReg(Y.loc[:,~Y.columns.duplicated()].groupby("pyro2").mean().reset_index().iloc[:,[0,19]+list(range(1,19))].iloc[:,:11]).X
                        X=pd.concat([pd.DataFrame(dfXY[["pyro2"]+ylist].reset_index(drop=True)),func_avg_zone(x2list).iloc[:,[0,1,2,3,4,5,6,7,8,9]]],axis=1)
                        X=func_get_dataReg(X.loc[:,~X.columns.duplicated()].groupby("pyro2").mean().reset_index().iloc[:,[0,19]+list(range(1,19))].iloc[:,[0,1]+list(range(11,20))]).X
                    else:#pyro2
                        Y=pd.concat([pd.DataFrame(dfXY[["pyro2"]+ylist].reset_index(drop=True)),func_avg_zone(x2list).iloc[:,[0,1,2,3,4,5,6,7,8,9]]],axis=1)
                        Y=Y.loc[:,~Y.columns.duplicated()].groupby("pyro2").mean().reset_index().loc[:,ylist[0]]
                        X=pd.concat([pd.DataFrame(dfXY[["pyro2"]+ylist].reset_index(drop=True)),func_avg_zone(x2list).iloc[:,[0,1,2,3,4,5,6,7,8,9]]],axis=1)
                        X=func_get_dataReg(X.loc[:,~X.columns.duplicated()].groupby("pyro2").mean().reset_index().iloc[:,:]).X
                    sns.regplot(x=X,y=Y,ax=axes[r,c],color='black',line_kws={'lw':2,'color':'red'},order=1,scatter=True,marker='x',scatter_kws={'s':40},x_ci=None,ci=None)#,marker='x',scatter_kws={'s':20}   
                    if (len(X.unique())>1) & (len(Y.unique())>1):
                        nan = np.logical_or(Y.isna().to_numpy(), X.isna().to_numpy())#OR of na values in X and Y, as pearsonr doesnt like nans
                        axes[r,c].legend(['r\u00b2={:.2f}, p={:.2E}'.format(st.pearsonr(Y[~nan],X[~nan])[0]**2,st.pearsonr(Y[~nan],X[~nan])[1])],fontsize='medium')
                    else: axes[r,c].legend([hue[3]],loc='best')
                axes[r,c].set_title(label='{}'.format(tot[t]),loc='center',pad=10,fontsize=17)
                axes[r,c].set_xlabel(xlabel=tot[t],labelpad=10,fontsize='x-large')
                axes[r,c].set_ylabel(ylabel=ylabel,labelpad=10,fontsize='x-large')
                axes[r,c].tick_params(axis='x', labelsize=15)
                axes[r,c].tick_params(axis='y', labelsize=15)
                #axes[r,c].legend([hue[3]],loc='best')#,loc='best',hue[3]
                #axes.legend(zoneTemp_vars_series[~zoneTemp_vars_series.str.contains(".*CV.*|.*Setpoint.*",regex=True)].tolist())#If not hue, show by zone
                t+=1
                
            #remove unused axes
            #print(len(axes[r,c].get_children()))
            if len(axes[r,c].get_children()) <=10:#empty plots have 10 children
                axes[r,c].set_visible(False)
                
    fig.tight_layout(pad=3)
    sns.set_style("darkgrid")
    
    plt.savefig(r"C:\Users\js5296\Desktop\NAS\Projects\AP1_Furnace_Control\EDAs\new\MV\newTry{}.png".format(title), bbox_inches="tight")
#

# #ZoneTempFdbk vs zone AirFlow: avg Bottom/Top
# varAirSP=zoneAirGas_vars_series[np.logical_and(zoneAirGas_vars_series.str.contains("AirFlow.*Set[Pp]oint.*",regex=True), ~zoneAirGas_vars_series.str.contains(".*ControlValve.*",regex=True))].tolist()
# varAir=zoneAirGas_vars_series[np.logical_and(zoneAirGas_vars_series.str.contains("AirFlow",regex=False), ~zoneAirGas_vars_series.str.contains(".*ControlValve.*|.*Set[Pp]oint.*",regex=True))].tolist()
# Temp=zoneTemp_vars_series[np.logical_and(~zoneTemp_vars_series.str.contains("CV",regex=False), ~zoneTemp_vars_series.str.contains(".*S.*[Pp].*",regex=True))].tolist()#9
# tot=["Total AirFlowSP","Total AirFlowFdbk"]
# col=3
# row=7
# 
# func_plot_reg(ylist=Temp,ylabel="ZoneTempFdbk",x1list=varAirSP,x1label="AirFlowSP",x2list=varAir,x2label="AirFlow",nrow=row,ncol=col,Zone=True,totlist=tot,tile="scatter")

### pyro2goal vs pyro2

In [ ]:
x=["AP1_FurnacePyro2Temp"]
y=["pyro2"]
col=2
row=2

func_plot_reg(ylist=y,ylabel="pyro2",x1list=x,x1label="AP1_FurnacePyro2Temp",nrow=row,ncol=col,title="Pyro2Goal_vs_Fdbk")

dfXY[["pyro2","AP1_FurnacePyro2Temp"]].groupby("pyro2").mean().reset_index()

### Pyro2 vs ZoneTemp, ZoneAir/Gas, Recuperator, RestFurnace, CSdeltaTmin, rampFdbk, soakFdbk

In [ ]:
#pyro2 vs ZoneTemp
sns.set_style("darkgrid")

#var=zoneTemp_vars_series[~zoneTemp_vars_series.str.contains(".*CV.*|.*Setpoint.*",regex=True)].tolist()


var=zoneTemp_vars_series[~zoneTemp_vars_series.str.contains(".*CV.*",regex=True)].tolist()
v=0
tot=["ZoneFdbk","ZoneSetpoints"]
t=0
col=3
row=(len(var)//col+1)+2#+2 for the 2 scatterplots of SP/Fdbk of all zones integrated
hue = ["grade","TurnDownOnCompressed","WidthFamCluster","SteelFamCluster"]

fig,axes = plt.subplots(row,col,figsize=(20,(row*20/5)+2),facecolor='white')
#fig(facecolor='black')
fig.suptitle("Pyro2 fdbk vs ZoneTemp by {}".format(hue[3]),fontsize=30)
for r in range(row):
    for c in range(col):
        if v<len(var):
            data=dfXY[["pyro2","AP1_FurnacePyro2Temp"]+[var[v]]].groupby("pyro2").mean().reset_index()
            sns.scatterplot(dfXY,x=var[v],y="AP1_FurnacePyro2Temp",ax=axes[r,c],hue=hue[3],palette="pastel")#,size=20
            sns.kdeplot(data=dfXY,x=var[v],y="AP1_FurnacePyro2Temp",levels=5,thresh=0.2,alpha=0.5,hue=hue[3],ax=axes[r,c],palette=["green","purple"])#,levels=5,thresh=0.2,palette="dark:#5A9_r
            sns.regplot(data=data,x=var[v],y="AP1_FurnacePyro2Temp",ax=axes[r,c],color='black',line_kws={'lw':2,'color':'fuchsia'},order=1,scatter=True,marker='x',scatter_kws={'s':40})#,marker='x',scatter_kws={'s':20}
            #axes[r,c].set_title(label='Pyro2 vs {} by {}'.format(var[v],hue[3]),loc='center',pad=10,fontsize=20)
            axes[r,c].set_title(label='{}'.format(var[v]),loc='center',pad=10,fontsize=17)
            axes[r,c].set_xlabel(xlabel=var[v],labelpad=10,fontsize='x-large')
            axes[r,c].set_ylabel(ylabel="AP1_FurnacePyro2Temp",labelpad=10,fontsize='x-large')
            nan = np.logical_or(data["AP1_FurnacePyro2Temp"].isna().to_numpy(), data[var[v]].isna().to_numpy())#OR of na values in X and Y, as pearsonr doesnt like nans
            axes[r,c].legend(['r\u00b2={:.2f}, p={:.2E}'.format(st.pearsonr(data["AP1_FurnacePyro2Temp"][~nan],data[var[v]][~nan])[0]**2,st.pearsonr(data["AP1_FurnacePyro2Temp"][~nan],data[var[v]][~nan])[1])],fontsize='medium')
            axes[r,c].tick_params(axis='x', labelsize=15)
            axes[r,c].tick_params(axis='y', labelsize=15)
            #axes[r,c].set_facecolor('black')
            v+=1
        elif (v>=len(var)) & (t<len(tot)):#Print SP/Fdbk zones alltoguether
            if t==0:
                data=dfXY[["pyro2","AP1_FurnacePyro2Temp"]+zoneTemp_vars_series[~zoneTemp_vars_series.str.contains(".*CV.*|.*Setpoint.*",regex=True)].tolist()].groupby("pyro2").mean().reset_index()
                for variable in zoneTemp_vars_series[~zoneTemp_vars_series.str.contains(".*CV.*|.*Setpoint.*",regex=True)].tolist():
                    sns.scatterplot(dfXY,x=variable,y="AP1_FurnacePyro2Temp",ax=axes[r,c],hue=hue[3])#,size=2
                
                sns.regplot(data=func_get_dataReg(data),x='X',y="AP1_FurnacePyro2Temp",ax=axes[r,c],color='black',line_kws={'lw':2,'color':'red'},order=1,scatter=True,marker='x',scatter_kws={'s':40})#,marker='x',scatter_kws={'s':20}   
                nan = np.logical_or(func_get_dataReg(data)["AP1_FurnacePyro2Temp"].isna().to_numpy(), func_get_dataReg(data)['X'].isna().to_numpy())#OR of na values in X and Y, as pearsonr doesnt like nans
                axes[r,c].legend(['r\u00b2={:.2f}, p={:.2E}'.format(st.pearsonr(func_get_dataReg(data)["AP1_FurnacePyro2Temp"][~nan],func_get_dataReg(data)['X'][~nan])[0]**2,st.pearsonr(func_get_dataReg(data)["AP1_FurnacePyro2Temp"][~nan],func_get_dataReg(data)['X'][~nan])[1])],fontsize='medium')
            elif t==1:
                data=dfXY[["pyro2","AP1_FurnacePyro2Temp"]+zoneTemp_vars_series[zoneTemp_vars_series.str.contains(".*Setpoint.*",regex=True)].tolist()].groupby("pyro2").mean().reset_index()
                for variable in zoneTemp_vars_series[zoneTemp_vars_series.str.contains(".*Setpoint.*",regex=True)].tolist():
                    sns.scatterplot(dfXY,x=variable,y="AP1_FurnacePyro2Temp",ax=axes[r,c],hue=hue[3])#,size=2
                
                sns.regplot(data=func_get_dataReg(data),x='X',y="AP1_FurnacePyro2Temp",ax=axes[r,c],color='black',line_kws={'lw':2,'color':'red'},order=1,scatter=True,marker='x',scatter_kws={'s':40})#,marker='x',scatter_kws={'s':20}   
                nan = np.logical_or(func_get_dataReg(data)["AP1_FurnacePyro2Temp"].isna().to_numpy(), func_get_dataReg(data)['X'].isna().to_numpy())#OR of na values in X and Y, as pearsonr doesnt like nans
                axes[r,c].legend(['r\u00b2={:.2f}, p={:.2E}'.format(st.pearsonr(func_get_dataReg(data)["AP1_FurnacePyro2Temp"][~nan],func_get_dataReg(data)['X'][~nan])[0]**2,st.pearsonr(func_get_dataReg(data)["AP1_FurnacePyro2Temp"][~nan],func_get_dataReg(data)['X'][~nan])[1])],fontsize='medium')
                
            axes[r,c].set_title(label='{}'.format(tot[t]),loc='center',pad=10,fontsize=17)
            axes[r,c].set_xlabel(xlabel=tot[t],labelpad=10,fontsize='x-large')
            axes[r,c].set_ylabel(ylabel="AP1_FurnacePyro2Temp",labelpad=10,fontsize='x-large')
            axes[r,c].tick_params(axis='x', labelsize=15)
            axes[r,c].tick_params(axis='y', labelsize=15)
            #axes[r,c].legend([hue[3]],loc='best')#,loc='best',hue[3]
            #axes.legend(zoneTemp_vars_series[~zoneTemp_vars_series.str.contains(".*CV.*|.*Setpoint.*",regex=True)].tolist())#If not hue, show by zone
            t+=1
            
        #remove unused axes
        #print(len(axes[r,c].get_children()))
        if len(axes[r,c].get_children()) <=10:#empty plots have 10 children
            axes[r,c].set_visible(False)
            
fig.tight_layout(pad=3)
#sns.set(rc={'axes.facecolor':'dark', 'figure.facecolor':'dark'})

plt.savefig(r"C:\Users\js5296\Desktop\NAS\Projects\AP1_Furnace_Control\EDAs\new\MV\newPyro2_vs_ZoneTemp.png", bbox_inches="tight")

In [ ]:
dfXY["AP1_FurnaceZone3TemperatureSetpoint"][dfXY["AP1_FurnaceZone3TemperatureSetpoint"].between(1000,1400)].shape

sns.scatterplot(data=func_avg_zone(varRatioSP),x="Zone0",y="AP1_FurnacePyro2Temp")#,size=20,hue=hue[3]
zone[0]
func_avg_zone(varRatioSP)[hue[3]]
dfXY[hue[3]]

In [ ]:
#Pyro2 vs zone Ratio: avg Bottom/Top
varRatioSP=zoneAirGas_vars_series[np.logical_and(zoneAirGas_vars_series.str.contains(".*Ratio.*S.*[Pp].*",regex=True), ~zoneAirGas_vars_series.str.contains(".*ControlValve.*",regex=True))].tolist()
varRatio=zoneAirGas_vars_series[np.logical_and(zoneAirGas_vars_series.str.contains(".*Ratio.*",regex=True), ~zoneAirGas_vars_series.str.contains(".*ControlValve.*|.*Ratio.*S.*[Pp].*",regex=True))].tolist()
tot=["ZoneSetpoints","ZoneFdbk"]
col=3
row=7

func_plot_reg(ylist=["AP1_FurnacePyro2Temp"],ylabel="pyro2",x1list=varRatioSP,x1label="RatioSP",x2list=varRatio,x2label="RatioFdbk",nrow=row,ncol=col,Zone=True,Avg=True,totlist=tot,title="Pyro2_vs_ZoneRatio")

zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains(".*Gas.*ControlValve.*",regex=True)]
dfXY.iloc[:1,[1,0]]
pd.concat([pd.DataFrame(dfXY["pyro2"].reset_index(),columns=["pyro2"]),func_avg_zone(varRatioSP).iloc[:,[9,0,1,2,3,4,5,6,7,8]]],axis=1).groupby("pyro2").mean().reset_index()
#func_avg_zone(varRatioSP).iloc[:,[9,0,1,2,3,4,5,6,7,8]]

In [ ]:
#Pyro2 vs zone CV: avg Bottom/Top
varCVair=zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains(".*Air.*ControlValve.*",regex=True)].tolist()
varCVgas=zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains(".*Gas.*ControlValve.*",regex=True)].tolist()
tot=["Total AirCV","Total GasCV"]
col=3
row=7

func_plot_reg(ylist=["AP1_FurnacePyro2Temp"],ylabel="pyro2",x1list=varCVair,x1label="AirCV",x2list=varCVgas,x2label="GasCV",nrow=row,ncol=col,Zone=True,Avg=True,totlist=tot,title="Pyro2_vs_ZoneCV")

#func_avg_zone(varCVair).loc[(func_avg_zone(varCVair)["AP1_FurnacePyro2Temp"]<0).values,["Zone2","AP1_FurnacePyro2Temp"]]
##(func_avg_zone(varCVair)["AP1_FurnacePyro2Temp"]<0).values.astype(bool)
##func_avg_zone(varCVair).loc[[(func_avg_zone(varCVair)["AP1_FurnacePyro2Temp"]<0).],["Zone2","AP1_FurnacePyro2Temp"]]
#pd.concat([pd.DataFrame(dfXY["pyro2"].reset_index(),columns=["pyro2"]),func_avg_zone(varCVair).iloc[:,[9,0,1,2,3,4,5,6,7,8]]],axis=1).groupby("pyro2").mean().reset_index()
#func_avg_zone(varCVair)
#dfXY[["pyro2","AP1_FurnacePyro2Temp","AP1_FurnaceZone2BottomAirFlowControlValveOutput","AP1_FurnaceZone2TopAirFlowControlValveOutput"]].groupby("pyro2").mean().reset_index()
#dfXY[["AP1_FurnaceZone2BottomAirFlowControlValveOutput","AP1_FurnaceZone2TopAirFlowControlValveOutput"]].mean(axis=1)==(dfXY["AP1_FurnaceZone2BottomAirFlowControlValveOutput"]+dfXY["AP1_FurnaceZone2TopAirFlowControlValveOutput"])/2
#(dfXY["AP1_FurnaceZone2BottomAirFlowControlValveOutput"]+dfXY["AP1_FurnaceZone2TopAirFlowControlValveOutput"])/2
##func_avg_zone(varCVair)
#varCVair[0+1]

df=pd.DataFrame(np.zeros((dfXY.shape[0],len(varCVair)//2)))
df.iloc[:,:]=np.nan
i=0
j=0
while i < len(varCVair)//2:#j<9
    df.iloc[:,i]=dfXY.loc[:,[varCVair[j],varCVair[j+1]]].mean(axis=1)
    #print("{} vs {}".format(varCVair[j],varCVair[j+1]))
    j+=2
    i+=1
(~(df.iloc[:,1]==((dfXY["AP1_FurnaceZone1BottomAirFlowControlValveOutput"]+dfXY["AP1_FurnaceZone1TopAirFlowControlValveOutput"])/2).reset_index(drop=True))).sum()

In [ ]:
#Pyro2 vs zone AirFlow: avg Bottom/Top
varAirSP=zoneAirGas_vars_series[np.logical_and(zoneAirGas_vars_series.str.contains("AirFlow.*Set[Pp]oint.*",regex=True), ~zoneAirGas_vars_series.str.contains(".*ControlValve.*",regex=True))].tolist()
varAir=zoneAirGas_vars_series[np.logical_and(zoneAirGas_vars_series.str.contains("AirFlow",regex=False), ~zoneAirGas_vars_series.str.contains(".*ControlValve.*|.*Set[Pp]oint.*",regex=True))].tolist()
tot=["Total AirFlowSP","Total AirFlowFdbk"]
col=3
row=7

func_plot_reg(ylist=["AP1_FurnacePyro2Temp"],ylabel="pyro2",x1list=varAirSP,x1label="AirSP",x2list=varAir,x2label="AirFdbk",nrow=row,ncol=col,Zone=True,Avg=True,totlist=tot,title="Pyro2_vs_ZoneAirFlow")

In [ ]:
#Pyro2 vs zone GasFlow: avg Bottom/Top
varGas=zoneAirGas_vars_series[np.logical_and(zoneAirGas_vars_series.str.contains("GasFlow",regex=False), ~zoneAirGas_vars_series.str.contains("ControlValve",regex=False))].tolist()
tot=["Total Gas"]
col=3
row=7

func_plot_reg(ylist=["AP1_FurnacePyro2Temp"],ylabel="pyro2",x1list=varGas,x1label="AirSP",nrow=row,ncol=col,Zone=True,Avg=True,totlist=tot,title="Pyro2_vs_ZoneGasFlow")

rest_vars_series
dfXY[["pyro2","AP1_FurnacePyro2Temp","AP1_FurnacePressureCV"]].groupby("pyro2").mean()
#dfXY["AP1_FurnacePressureCV"].min()
dfXY[["pyro2","AP1_FurnacePyro2Temp"]+var].groupby("pyro2").mean().reset_index()
rest_vars_series

In [ ]:
#pyro2 vs restVars
var=rest_vars_series.tolist()+["CSdeltaTmin","rampFdbk","soakFdbk"]
y=["AP1_FurnacePyro2Temp"]
col=3
row=7

func_plot_reg(ylist=y,ylabel="pyro2",x1list=var,nrow=row,ncol=col,title="Pyro2_vs_RestFurnace",kde=True)

proceso_vars_series

In [ ]:
#pyro2 vs proceso vars
var=proceso_vars_series.tolist()
y=["AP1_FurnacePyro2Temp"]
col=3
row=7

func_plot_reg(ylist=y,ylabel="pyro2",x1list=var,nrow=row,ncol=col,title="Pyro2_vs_Proceso")

recuperator_vars_series
#dfXY[["pyro2","AP1_FurnacePyro2Temp"]+var].groupby("pyro2").mean().reset_index()

In [ ]:
#pyro2 vs recuperator vars
var=recuperator_vars_series.tolist()
y=["AP1_FurnacePyro2Temp"]
col=3
row=7

func_plot_reg(ylist=y,ylabel="pyro2",x1list=var,nrow=row,ncol=col,title="Pyro2_vs_Recuperator",kde=True)

### pyro2 vs pyro1

In [ ]:
x=["AP1_FurnacePyro1Temp"]
y=["AP1_FurnacePyro2Temp"]
col=2
row=2

func_plot_reg(ylist=y,ylabel="pyro2",x1list=x,x1label="pyro1",nrow=row,ncol=col,title="Pyro2_vs_Pyro1")

### ZoneTempSP vs ZoneTempFdbk

In [ ]:
#ZoneTempSP vs zone ZoneTempFdbk: avg Bottom/Top
TempSP=zoneTemp_vars_series[np.logical_and(~zoneTemp_vars_series.str.contains("CV",regex=False), zoneTemp_vars_series.str.contains(".*S.*[Pp].*",regex=True))].tolist()#10
vSP=0
Temp=zoneTemp_vars_series[np.logical_and(~zoneTemp_vars_series.str.contains("CV",regex=False), ~zoneTemp_vars_series.str.contains(".*S.*[Pp].*",regex=True))].tolist()#9
v=0
zone=["Zone0","Zone1","Zone2","Zone3","Zone4","Zone5","Zone6","Zone7","Zone8"]
z=0
col=3
row=7
#row=(len(zone)//col+1)+2#+2 for the 2 scatterplots of SP/Fdbk of all zones integrated
hue = ["grade","TurnDownOnCompressed","WidthFamCluster","SteelFamCluster"]
tot=True
fig,axes = plt.subplots(row,col,figsize=(20,(row*20/5)+2),facecolor='white')
fig.suptitle("TempSP vs TempFdbk by {}".format(hue[3]),fontsize=30)
for r in range(row):
    for c in range(col):
        if z==0:
            data=dfXY[["pyro2"]+TempSP+Temp].groupby("pyro2").mean().reset_index()
            sns.scatterplot(data=dfXY,x=Temp[v],y=TempSP[vSP],ax=axes[r,c],hue=hue[3],palette="pastel")#,size=20
            sns.kdeplot(data=dfXY,x=Temp[v],y=TempSP[vSP],levels=5,thresh=0.2,alpha=0.5,hue=hue[3],ax=axes[r,c],palette=["green","purple"])#,levels=5,thresh=0.2,palette="dark:#5A9_r
            sns.regplot(data=data,x=Temp[v],y=TempSP[vSP],ax=axes[r,c],color='pink',line_kws={'lw':2,'color':'red'},order=1,scatter=True,marker='x',scatter_kws={'s':40},x_ci=None,ci=None)#,marker='x',scatter_kws={'s':20}
            #axes[r,c].set_title(label='Pyro2 vs {} by {}'.format(var[v],hue[3]),loc='center',pad=10,fontsize=20)
            axes[r,c].set_title(label='Temp SP vs Fdbk in {}'.format(zone[z]),loc='center',pad=10,fontsize=17)
            axes[r,c].set_xlabel(xlabel=Temp[v],labelpad=10,fontsize='x-large')
            axes[r,c].set_ylabel(ylabel=TempSP[vSP],labelpad=10,fontsize='x-large')
            nan = np.logical_or(data[TempSP[vSP]].isna().to_numpy(), data[Temp[v]].isna().to_numpy())#OR of na values in X and Y, as pearsonr doesnt like nans
            axes[r,c].legend(['r\u00b2={:.2f}, p={:.2E}'.format(st.pearsonr(data[TempSP[vSP]][~nan],data[Temp[v]][~nan])[0]**2,st.pearsonr(data[TempSP[vSP]][~nan],data[Temp[v]][~nan])[1])],fontsize='large') 
            axes[r,c].tick_params(axis='x', labelsize=15)
            axes[r,c].tick_params(axis='y', labelsize=15)
            vSP+=1
            if vSP==2:
                v+=1
                z+=1
        elif z<len(zone):
            data=dfXY[["pyro2"]+TempSP+Temp].groupby("pyro2").mean().reset_index()
            sns.scatterplot(data=dfXY,x=Temp[v],y=TempSP[vSP],ax=axes[r,c],hue=hue[3],palette="pastel")#,size=20
            sns.kdeplot(data=dfXY,x=Temp[v],y=TempSP[vSP],levels=5,thresh=0.2,alpha=0.5,hue=hue[3],ax=axes[r,c],palette=["green","purple"])#,levels=5,thresh=0.2,palette="dark:#5A9_r
            sns.regplot(data=data,x=Temp[v],y=TempSP[vSP],ax=axes[r,c],color='pink',line_kws={'lw':2,'color':'red'},order=1,scatter=True,marker='x',scatter_kws={'s':40},x_ci=None,ci=None)#,marker='x',scatter_kws={'s':20}
            #axes[r,c].set_title(label='Pyro2 vs {} by {}'.format(var[v],hue[3]),loc='center',pad=10,fontsize=20)
            axes[r,c].set_title(label='Temp SP vs Fdbk in {}'.format(zone[z]),loc='center',pad=10,fontsize=17)
            axes[r,c].set_xlabel(xlabel=Temp[v],labelpad=10,fontsize='x-large')
            axes[r,c].set_ylabel(ylabel=TempSP[vSP],labelpad=10,fontsize='x-large')
            nan = np.logical_or(data[TempSP[vSP]].isna().to_numpy(), data[Temp[v]].isna().to_numpy())#OR of na values in X and Y, as pearsonr doesnt like nans
            axes[r,c].legend(['r\u00b2={:.2f}, p={:.2E}'.format(st.pearsonr(data[TempSP[vSP]][~nan],data[Temp[v]][~nan])[0]**2,st.pearsonr(data[TempSP[vSP]][~nan],data[Temp[v]][~nan])[1])],fontsize='large') 
            axes[r,c].tick_params(axis='x', labelsize=15)
            axes[r,c].tick_params(axis='y', labelsize=15)
            vSP+=1
            v+=1
            z+=1 
            
        elif (z>=len(zone)) & tot:#Print SP/Fdbk zones alltoguether
            v=0
            vSP=0
            for variable in zone:
                if variable == "Zone0":
                    while vSP<2:
                        sns.scatterplot(data=dfXY,x=Temp[v],y=TempSP[vSP],ax=axes[r,c],hue=hue[3])#,size=2
                        #print("{} vs {}".format(TempSP[vSP],Temp[v]))
                        vSP+=1
                    if vSP==2:
                        v+=1
                else:
                    sns.scatterplot(data=dfXY,x=Temp[v],y=TempSP[vSP],ax=axes[r,c],hue=hue[3])#,size=2
                    #print("{} vs {}".format(TempSP[vSP],Temp[v]))
                    v+=1
                    vSP+=1
                    
            Y=func_get_dataReg(dfXY[["pyro2","AP1_FurnacePyro2Temp"]+TempSP].groupby("pyro2").mean().reset_index()).X
            X=func_get_dataReg(dfXY[["pyro2","AP1_FurnacePyro2Temp"]+Temp].groupby("pyro2").mean().reset_index()).X
            sns.regplot(x=X,y=Y,ax=axes[r,c],color='black',line_kws={'lw':2,'color':'red'},order=1,scatter=True,marker='x',scatter_kws={'s':40},x_ci=None,ci=None)#,marker='x',scatter_kws={'s':20}   
            nan = np.logical_or(X.isna().to_numpy(), Y.isna().to_numpy())#OR of na values in X and Y, as pearsonr doesnt like nans
            axes[r,c].legend(['r\u00b2={:.2f}, p={:.2E}'.format(st.pearsonr(Y[~nan],X[~nan])[0]**2,st.pearsonr(Y[~nan],X[~nan])[1])],fontsize='medium')
            axes[r,c].set_title(label='Total Zones Temp',loc='center',pad=10,fontsize=17)
            axes[r,c].set_xlabel(xlabel="Temp Fdbk",labelpad=10,fontsize='x-large')
            axes[r,c].set_ylabel(ylabel="Temp SP",labelpad=10,fontsize='x-large')
            axes[r,c].tick_params(axis='x', labelsize=15)
            axes[r,c].tick_params(axis='y', labelsize=15)
            #axes[r,c].legend([hue[3]],loc='best')#,loc='best',hue[3]
            #axes.legend(zoneTemp_vars_series[~zoneTemp_vars_series.str.contains(".*CV.*|.*Setpoint.*",regex=True)].tolist())#If not hue, show by zone
            tot=False
            
        #remove unused axes
        #print(len(axes[r,c].get_children()))
        if len(axes[r,c].get_children()) <=10:#empty plots have 10 children
            axes[r,c].set_visible(False)
            
fig.tight_layout(pad=3)
sns.set_style("darkgrid")

plt.savefig(r"C:\Users\js5296\Desktop\NAS\Projects\AP1_Furnace_Control\EDAs\new\MV\newZoneTempSP_vs_ZoneTempFdbk.png", bbox_inches="tight")

zoneTemp_vars_series
zoneTemp_vars_series[np.logical_and(~zoneTemp_vars_series.str.contains("CV",regex=False), zoneTemp_vars_series.str.contains(".*S.*[Pp].*",regex=True))]
func_get_dataReg(dfXY[["pyro2","AP1_FurnacePyro2Temp"]+TempSP].groupby("pyro2").mean().reset_index())

### ZoneTempFdbk vs ZoneAirGasFlows,Recuperator,RestFurnaceVars,Proceso

In [ ]:
#ZoneTempFdbk vs zone AirFlow: avg Bottom/Top
varAirSP=zoneAirGas_vars_series[np.logical_and(zoneAirGas_vars_series.str.contains("AirFlow.*Set[Pp]oint.*",regex=True), ~zoneAirGas_vars_series.str.contains(".*ControlValve.*",regex=True))].tolist()
varAir=zoneAirGas_vars_series[np.logical_and(zoneAirGas_vars_series.str.contains("AirFlow",regex=False), ~zoneAirGas_vars_series.str.contains(".*ControlValve.*|.*Set[Pp]oint.*",regex=True))].tolist()
Temp=zoneTemp_vars_series[np.logical_and(~zoneTemp_vars_series.str.contains("CV",regex=False), ~zoneTemp_vars_series.str.contains(".*S.*[Pp].*",regex=True))].tolist()#9
tot=["Total AirFlowSP","Total AirFlowFdbk"]
col=3
row=7

func_plot_reg(ylist=Temp,ylabel="ZoneTempFdbk",x1list=varAirSP,x1label="AirSP",x2list=varAir,x2label="AirFdbk",nrow=row,ncol=col,Zone=True,Avg=True,totlist=tot,title="ZoneTempFdbk_vs_AirFlow")

func_avg_zone(varAirSP)
pd.concat([pd.DataFrame(dfXY[["pyro2"]+Temp].reset_index(drop=True)),func_avg_zone(varAir).iloc[:,[0,1,2,3,4,5,6,7,8,9]]],axis=1).groupby("pyro2").mean().reset_index().iloc[:,[0,19]+list(range(1,19))]#.iloc[:,[0,1]+list(range(11,20))]
func_get_dataReg(pd.concat([pd.DataFrame(dfXY[["pyro2"]+Temp].reset_index(drop=True)),func_avg_zone(varAirSP).iloc[:,[0,1,2,3,4,5,6,7,8,9]]],axis=1).groupby("pyro2").mean().reset_index().iloc[:,[0,19]+list(range(1,19))].iloc[:,:11]).X
func_get_dataReg(pd.concat([pd.DataFrame(dfXY[["pyro2"]+Temp].reset_index(drop=True)),func_avg_zone(varAirSP).iloc[:,[0,1,2,3,4,5,6,7,8,9]]],axis=1).groupby("pyro2").mean().reset_index().iloc[:,[0,19]+list(range(1,19))].iloc[:,[0,1]+list(range(11,20))]).X
#pd.concat([pd.DataFrame(dfXY[Temp+["pyro2"]].reset_index(drop=True)),func_avg_zone(varAirSP).iloc[:,:]],axis=1)

In [ ]:
#ZoneTempFdbk vs zone GasFlow: avg Bottom/Top
varGas=zoneAirGas_vars_series[np.logical_and(zoneAirGas_vars_series.str.contains("GasFlow",regex=False), ~zoneAirGas_vars_series.str.contains("ControlValve",regex=False))].tolist()
Temp=zoneTemp_vars_series[np.logical_and(~zoneTemp_vars_series.str.contains("CV",regex=False), ~zoneTemp_vars_series.str.contains(".*S.*[Pp].*",regex=True))].tolist()#9
tot=["Total GasFlowFdbk"]
col=3
row=7

func_plot_reg(ylist=Temp,ylabel="ZoneTempFdbk",x1list=varGas,x1label="GasFlow",nrow=row,ncol=col,Zone=True,Avg=True,totlist=tot,title="ZoneTempFdbk_vs_GasFlow")

In [ ]:
#ZoneTempFdbk vs zone CV: avg Bottom/Top
varCVair=zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains(".*Air.*ControlValve.*",regex=True)].tolist()
varCVgas=zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains(".*Gas.*ControlValve.*",regex=True)].tolist()
Temp=zoneTemp_vars_series[np.logical_and(~zoneTemp_vars_series.str.contains("CV",regex=False), ~zoneTemp_vars_series.str.contains(".*S.*[Pp].*",regex=True))].tolist()#9
tot=["Total AirCV","Total GasCV"]
col=3
row=7

func_plot_reg(ylist=Temp,ylabel="ZoneTempFdbk",x1list=varCVair,x1label="CVair",x2list=varCVgas,x2label="CVgas",nrow=row,ncol=col,Zone=True,Avg=True,totlist=tot,title="ZoneTempFdbk_vs_CV",kde=True)

In [ ]:
#ZoneTempFdbk vs zone Ratio: avg Bottom/Top
varRatioSP=zoneAirGas_vars_series[np.logical_and(zoneAirGas_vars_series.str.contains(".*Ratio.*S.*[Pp].*",regex=True), ~zoneAirGas_vars_series.str.contains(".*ControlValve.*",regex=True))].tolist()
varRatio=zoneAirGas_vars_series[np.logical_and(zoneAirGas_vars_series.str.contains(".*Ratio.*",regex=True), ~zoneAirGas_vars_series.str.contains(".*ControlValve.*|.*Ratio.*S.*[Pp].*",regex=True))].tolist()
Temp=zoneTemp_vars_series[np.logical_and(~zoneTemp_vars_series.str.contains("CV",regex=False), ~zoneTemp_vars_series.str.contains(".*S.*[Pp].*",regex=True))].tolist()#9
tot=["Total RatioSP","Total RatioFdbk"]
col=3
row=7

func_plot_reg(ylist=Temp,ylabel="ZoneTempFdbk",x1list=varRatioSP,x1label="RatioSP",x2list=varRatio,x2label="Ratio",nrow=row,ncol=col,Zone=True,Avg=True,totlist=tot,title="ZoneTempFdbk_vs_Ratio")

pd.concat([pd.DataFrame(dfXY[Temp+["pyro2"]].reset_index(drop=True)),func_avg_zone(varRatio).iloc[:,:]],axis=1)
#pd.concat([pd.DataFrame(dfXY[["pyro2"]+Temp].reset_index(drop=True)),func_avg_zone(varRatio).iloc[:,[0,1,2,3,4,5,6,7,8,9]]],axis=1).groupby("pyro2").mean().reset_index().iloc[:,[0,19]+list(range(1,19))]
#recuperator_vars_series
#func_get_dataReg(pd.concat([pd.DataFrame(dfXY[["pyro2"]+Temp].reset_index(drop=True)),func_avg_zone(varRatioSP).iloc[:,[0,1,2,3,4,5,6,7,8,9]]],axis=1).groupby("pyro2").mean().reset_index().iloc[:,[0,19]+list(range(1,19))].iloc[:,:11]).X
#func_get_dataReg(dfXY[["pyro2","AP1_FurnacePyro2Temp"]+Temp].groupby("pyro2").mean().reset_index())
pd.concat([pd.DataFrame(dfXY[Temp].mean(axis=1),columns=["Temp"]),dfXY[recuperator_vars_series.tolist()+["pyro2"]]],axis=1).iloc[:,[len(recuperator_vars_series)-1+2]+list(range(len(recuperator_vars_series)-1+2))].groupby("pyro2").mean().reset_index()
#pd.concat([pd.DataFrame(dfXY[Temp].mean(axis=1),columns=["Temp"]),dfXY[var+hue]],axis=1)
recuperator_vars_series

In [ ]:
#ZoneTempFdbk vs recuperator vars
var=recuperator_vars_series.tolist()
Temp=zoneTemp_vars_series[np.logical_and(~zoneTemp_vars_series.str.contains("CV",regex=False), ~zoneTemp_vars_series.str.contains(".*S.*[Pp].*",regex=True))].tolist()#9
col=3
row=7

func_plot_reg(ylist=Temp,ylabel="ZoneTempFdbk",x1list=var,nrow=row,ncol=col,title="ZoneTempFdbk_vs_Recuperator",kde=True)

recuperator_vars_series
rest_vars_series

In [ ]:
#ZoneTempFdbk vs rest furnace vars
var=rest_vars_series.tolist()+["CSdeltaTmin","rampFdbk","soakFdbk"]
Temp=zoneTemp_vars_series[np.logical_and(~zoneTemp_vars_series.str.contains("CV",regex=False), ~zoneTemp_vars_series.str.contains(".*S.*[Pp].*",regex=True))].tolist()#9
col=3
row=7

func_plot_reg(ylist=Temp,ylabel="ZoneTempFdbk",x1list=var,nrow=row,ncol=col,title="ZoneTempFdbk_vs_RestFurnace",kde=True)

proceso_vars_series

In [ ]:
#ZoneTempFdbk vs proceso vars
var=proceso_vars_series.tolist()
Temp=zoneTemp_vars_series[np.logical_and(~zoneTemp_vars_series.str.contains("CV",regex=False), ~zoneTemp_vars_series.str.contains(".*S.*[Pp].*",regex=True))].tolist()#9
col=3
row=7

func_plot_reg(ylist=Temp,ylabel="ZoneTempFdbk",x1list=var,nrow=row,ncol=col,title="ZoneTempFdbk_vs_Proceso")

### Multicollinearity with Variance Inflation Factor (VIF)

In [ ]:
def variance_inflation_factor(exog, exog_idx):
    """
    exog : ndarray, (nobs, k_vars)
        design matrix with all explanatory variables, as for example used in
        regression
    exog_idx : int
        index of the exogenous variable in the columns of exog
    """
    k_vars = exog.shape[1]
    x_i = exog[:, exog_idx]
    mask = np.arange(k_vars) != exog_idx
    x_noti = exog[:, mask]
    r_squared_i = OLS(x_i, x_noti, missing="drop").fit().rsquared
    vif = 1. / (1. - r_squared_i)
    return vif

def variance_inflation_factor_sklearn(exog, exog_idx):
    """
    exog : ndarray, (nobs, k_vars)
        design matrix with all explanatory variables, as for example used in
        regression
    exog_idx : int
        index of the exogenous variable in the columns of exog
    """
    k_vars = exog.shape[1]
    x_i = exog[:, exog_idx]
    mask = np.arange(k_vars) != exog_idx
    x_noti = exog[:, mask]
    r_squared_i = LinearRegression().fit(X=x_noti,y=x_i).score(X=x_noti,y=x_i)#OLS(x_i, x_noti, missing="drop").fit().rsquared
    vif = 1. / (1. - r_squared_i)
    return vif

In [ ]:
#Make binary categorical data int64
dfXY[dfXY.columns[(dfXY.dtypes == "category") & (~dfXY.columns.str.contains("grade")) & (~dfXY.columns.str.contains("SteelFamCluster"))]]=dfXY[dfXY.columns[(dfXY.dtypes == "category") & (~dfXY.columns.str.contains("grade")) & (~dfXY.columns.str.contains("SteelFamCluster"))]].astype(np.float64)
dfXY.dtypes

In [ ]:
#Make grade one-hot encoder: drop first to prevent from further multicollinearity among encoded vars
dfGradeEnc=pd.DataFrame(OneHotEncoder(sparse_output=False,drop='first').set_output(transform="pandas").fit_transform(pd.DataFrame(dfXY.grade)))#sparse_output=False,
dfXY=pd.merge(left=dfXY,right=dfGradeEnc,how="inner",left_index=True,right_index=True)
dfGrade=dfXY.grade.copy()
dfXY.drop(["grade"],axis=1,inplace=True)
dfXY.head(1)

In [ ]:
dfXY.SteelFamCluster = OrdinalEncoder().fit_transform(dfXY[["SteelFamCluster"]])
dfXY.head(1)

In [ ]:
print(dfXY.shape)
dfXY.dtypes

In [ ]:
#Make input df
dfX=dfXY.drop("pyro2",axis=1)
print(dfX.shape)
#add_constant(dfX,prepend=False,has_constant='add')

In [ ]:
#Compute VIF
vif_data=pd.DataFrame()
vif_data["feature"]=dfX.columns
vif_data["dtype"]=dfX.dtypes
vif_data["VIF_OLS"]=[variance_inflation_factor(add_constant(dfX,prepend=False,has_constant='add').values,i) for i in range(len(dfX.columns))]
vif_data["VIF_sklearn"]=[variance_inflation_factor_sklearn(dfX.dropna(axis=0).values,i) for i in range(len(dfX.columns))]
#vif_data.assign(dtype= lambda x: dfX.dtypes.loc[x.feature])
for i in range(vif_data.shape[0]):
    vif_data.iloc[i,1]=dfX.dtypes.loc[vif_data.feature[i]]
print(vif_data.shape)
vif_data.sort_values(by='VIF_sklearn',ascending=False,na_position='first')

In [ ]:
#Show Vif ordered
np.round(vif_data.loc[((vif_data.VIF_OLS>10) | (vif_data.VIF_sklearn>10) | (vif_data.VIF_sklearn.isin([np.inf,-np.inf]))),:],2).sort_values(by='VIF_sklearn',ascending=False,na_position='first')

In [ ]:
#Plot corr heatmap per var
plt.figure(figsize=(30,2))
# sns.set_context('paper', font_scale=1.4)
xcorr_df = dfX.corr(method='pearson',numeric_only=True)#method='spearman'
sns.heatmap(pd.DataFrame(xcorr_df['AP1_FurnaceTotalSummedAirFlow'].sort_values(ascending=False,key=np.abs)).T.map(lambda x: None if np.abs(x)<0.6 else x).dropna(axis=1), cmap='viridis')
#sns.heatmap(xcorr_df[['AP1_FurnaceTotalSummedAirFlow']].T, cmap='viridis')
plt.show()

In [ ]:
xcorr_df.iloc[:,0]
pd.DataFrame(xcorr_df['AP1_FurnaceTotalSummedAirFlow'].sort_values(ascending=False,key=np.abs)).T.map(lambda x: None if np.abs(x)<0.6 else x).dropna(axis=1)

In [ ]:
#Plot df corr heatmap
plt.figure(figsize=(30,60))
# sns.set_context('paper', font_scale=1.4)
xycorr_df = dfXY.corr(method='pearson',numeric_only=True).map(lambda x: None if np.abs(x)<0.8 else x)#method='spearman'
sns.heatmap(xycorr_df, cmap='viridis')
plt.title("INPUT corr>0.8 heatmap",fontsize=60,pad=50)
plt.show()

In [ ]:
dfXY.shape

In [ ]:
dfGroups=pd.DataFrame(dfXY_coils.loc[dfXY.index]).assign(group=lambda x: OrdinalEncoder().set_output(transform="pandas").fit_transform(x[["AP1_FurnaceCoilID"]]).astype(np.int64))#.value_counts()

In [ ]:
print(np.sum(~(dfGroups["AP1_FurnaceCoilID"].value_counts().values==dfGroups.group.value_counts().values)))

a=20.0
b=8.0
B=math.radians(20.0)
A=np.arcsin(a*math.sin(B)/b)
C=math.radians(180.0-math.degrees(B)-math.degrees(A))
ca=math.sin(C)*a/math.sin(A)
cb=math.sin(C)*b/math.sin(B)
print("A: ",math.degrees(A))
print("C: ",math.degrees(C))
print(ca,cb)